---
# Gmsh Tutorials
- this notebook aims to develop my skills using Gmsh Python API
- python-specific reference: https://gmsh.info/doc/texinfo/gmsh.html#x1


In [2]:
# Tutorial 1 - Geometry basics, elementary entities, physical groups
# https://gitlab.onelab.info/gmsh/gmsh/blob/gmsh_4_11_1/tutorials/python/t1.py
import gmsh
import sys

# Before using any functions in the Python API, Gmsh must be initialized:
if not gmsh.is_initialized():
    gmsh.initialize()

# Next we add a new model named "t1":
Name = "test_mesh"
gmsh.model.add(Name)


# 1.POINTS: The first type of `elementary entity' in Gmsh is a `Point'. To create a point
# the Python API function is gmsh.model.geo.addPoint()  or gmsh.model.geo.add_point()
# 3 arguments are the point coordinates (x, y, z)
# - the next (optional) argument is the target mesh size close to the point
# - the last (optional) argument is the point tag/identifier
lc = 1  # target mesh size
z = 0   # fixed z coordinate
gmsh.model.geo.addPoint(0, 0, z, lc, 1)
gmsh.model.geo.addPoint(0, 10, 0, lc, 2)
gmsh.model.geo.addPoint(10, 10, 0, lc, 3)
gmsh.model.geo.addPoint(10, 0, 0, lc, 4)

# 2.LINES: Curves are second type of elementery entities
# Straight line segments: the first 2 arguments are
# - point tags (the start and end points of the line)
# - the last (optional) is the line tag.
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)


# 3.SURFACES: to define a simple rectangular surface, curve loop (ordered list of connected curves)
# has to be defined.  Arguments are
# - list of curve tags
# - the curve loop tag (which must be unique
# amongst curve loops) as the second (optional) argument:
gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)

# add the surface as a list of curve loops: outline + holes(optional)
gmsh.model.geo.addPlaneSurface([1], 1)


# Before it can be meshed (and, more generally, before it can be used by API
# functions outside of the built-in CAD kernel functions), the CAD entities must
# be synchronized with the Gmsh model
gmsh.model.geo.synchronize()


# physical groups: mathematical ("domain", "boundary"), functional ("left wing", "fuselage") ...
# Here we define a physical curve that groups the left, right and bottom curves
# in a single group (with prescribed tag 5); and a physical surface with name
# "My surface" (with an automatic tag) containing the geometrical surface 1:
gmsh.model.addPhysicalGroup(1, [1, 3, 4], 5)              # curves physical group
gmsh.model.addPhysicalGroup(2, [1], name="My surface")  # surfaces physical group


# generate a 2D mesh
gmsh.model.mesh.generate(2)


# save it to disk (by default only physical groups are saved)
gmsh.write(f"./meshes/{Name}.msh")
gmsh.write(f"./meshes/{Name}.stl")

# By default, if physical groups are defined, Gmsh will export in
# the output mesh file only those elements that belong to at least one physical
# group. To force Gmsh to save all elements, you can use:
#    gmsh.option.setNumber("Mesh.SaveAll", 1)


# visualize the model in the GUI
gmsh.fltk.run()

# when finished with Gmsh Python API:
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000545319s, CPU 0.000909s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00770263s, CPU 0.007977s)
Info    : 143 nodes 288 elements
Info    : Writing './meshes/test_mesh.msh'...
Info    : Done writing './meshes/test_mesh.msh'
Info    : Writing './meshes/test_mesh.stl'...
Info    : Done writing './meshes/test_mesh.stl'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[cont

In [6]:
# Tutorial 2 - Transformations, extruded geometries, volumes

import gmsh
import sys
import numpy as np


# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

# add new model
Name = "test_mesh2"
gmsh.model.add(Name)

# setting up the geometry
lc = 1e-2  # target mesh size
gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
gmsh.model.geo.addPoint(.1, 0, 0, lc, 2)
gmsh.model.geo.addPoint(.1, .3, 0, lc, 3)
gmsh.model.geo.addPoint(0, .3, 0, lc, 4)
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(3, 2, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
gmsh.model.geo.addCurveLoop([4, 1, -2, 3], 1)
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [1, 2, 4], 5)  # left, right and bottom
gmsh.model.addPhysicalGroup(2, [1], name="My surface")  # surface group


#add new points and curves
gmsh.model.geo.addPoint(0, .4, 0, lc, 5)
gmsh.model.geo.addLine(4, 5, 5)


# transformation: translation
gmsh.model.geo.translate([(0, 5)], -0.02, 0, 0)  # translate([(dimension, tag)], dx, dy, dz)

# transformation: rotation
gmsh.model.geo.rotate([(0, 5)], 0, 0.3, 0, 0, 0, 1, -np.pi / 4)  # around (0, 0.3, 0) along the z axis by -Pi/4

# transformation: duplication
ov = gmsh.model.geo.copy([(0, 3)])        # copy([(dim, tag)]) -> returns (dim, tag)
gmsh.model.geo.translate(ov, 0, 0.05, 0)  # translation of the duplicated point


# creating lines to the duplicated point
gmsh.model.geo.addLine(3, ov[0][1], 7)
gmsh.model.geo.addLine(ov[0][1], 5, 8)
gmsh.model.geo.addCurveLoop([5, -8, -7, 3], 10)
gmsh.model.geo.addPlaneSurface([10], 11)

# duplicating and translating surfaces (surface 1 and surface 11)
ov = gmsh.model.geo.copy([(2, 1), (2, 11)])
gmsh.model.geo.translate(ov, 0.12, 0, 0)

print("New surfaces " + str(ov[0][1]) + " and " + str(ov[1][1]))  # printing the new tags

# Creating volumes 1: Manually
gmsh.model.geo.addPoint(0., 0.3, 0.12, lc, 100)
gmsh.model.geo.addPoint(0.1, 0.3, 0.12, lc, 101)
gmsh.model.geo.addPoint(0.1, 0.35, 0.12, lc, 102)

# accessing coordinates of points
gmsh.model.geo.synchronize()         # first synchronize the model
xyz = gmsh.model.getValue(0, 5, [])  # get coordinates of the point 5
xyz = gmsh.model.getValue(0, 5, [])  # get coordinates of the point 5
gmsh.model.geo.addPoint(xyz[0], xyz[1], 0.12, lc, 103)  # add point above 5

gmsh.model.geo.addLine(4, 100, 110)
gmsh.model.geo.addLine(3, 101, 111)
gmsh.model.geo.addLine(6, 102, 112)
gmsh.model.geo.addLine(5, 103, 113)
gmsh.model.geo.addLine(103, 100, 114)
gmsh.model.geo.addLine(100, 101, 115)
gmsh.model.geo.addLine(101, 102, 116)
gmsh.model.geo.addLine(102, 103, 117)

gmsh.model.geo.addCurveLoop([115, -111, 3, 110], 118)
gmsh.model.geo.addPlaneSurface([118], 119)
gmsh.model.geo.addCurveLoop([111, 116, -112, -7], 120)
gmsh.model.geo.addPlaneSurface([120], 121)
gmsh.model.geo.addCurveLoop([112, 117, -113, -8], 122)
gmsh.model.geo.addPlaneSurface([122], 123)
gmsh.model.geo.addCurveLoop([114, -110, 5, 113], 124)
gmsh.model.geo.addPlaneSurface([124], 125)
gmsh.model.geo.addCurveLoop([115, 116, 117, 114], 126)
gmsh.model.geo.addPlaneSurface([126], 127)

gmsh.model.geo.addSurfaceLoop([127, 119, 121, 123, 125, 11], 128)
gmsh.model.geo.addVolume([128], 129)



# Creating volumes 2: Extruding surface
ov2 = gmsh.model.geo.extrude([ov[1]], 0, 0, 0.12)  # extrude([(dim, tag), vector])


# setting up multiple mesh sizes with setSize()
gmsh.model.geo.mesh.setSize([(0, 103), (0, 105), (0, 109), (0, 102), (0, 28),
                             (0, 24), (0, 6), (0, 5)], lc * 3)


# synchronize with the model
gmsh.model.geo.synchronize()


# creating a physical group
gmsh.model.addPhysicalGroup(3, [129, 130], 1, "The volume")


# generate mesh:
gmsh.model.mesh.generate(3)

# save the mesh:
gmsh.write(f"./meshes/{Name}.msh")
gmsh.write(f"./meshes/{Name}.stl")
# gmsh.write(f"{Name}.geo_unrolled")  # `Gmsh Unrolled GEO' format


# visualize the model in the GUI
gmsh.fltk.run()

# when finished
gmsh.finalize()


New surfaces 12 and 17
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 30%] Meshing curve 13 (Line)
Info    : [ 30%] Meshing curve 14 (Line)
Info    : [ 30%] Meshing curve 15 (Line)
Info    : [ 40%] Meshing curve 16 (Line)
Info    : [ 40%] Meshing curve 18 (Line)
Info    : [ 40%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 50%] Meshing curve 110 (Line)
Info    : [ 50%] Meshing curve 111 (Line)
Info    : [ 60%] Meshing curve 112 (Line)
Info    : [ 60%] Meshing curve 113 (Line)
Info    : [ 60%] Meshing curve 114 (Line)
Info    : [ 70%] Meshing curve 115 (Line)
Info    : [ 70%] Meshing curve 116 (Line)
Info    : [ 70%] Meshing curve 117 (Line)
Info    : [ 80%] Meshing curve 131 (Line)
Info    

In [27]:
# Tutorial 3 - Extruded meshes, interactive (ONELAB) parameters, options

import gmsh
import numpy as np
import sys

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

def createGeometryAndMesh(Name):
    # Clear all models and create a new one
    gmsh.clear()
    gmsh.model.add(Name)

    # from tutorial 1
    lc = 0.01
    gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
    gmsh.model.geo.addPoint(.1, 0, 0, lc, 2)
    gmsh.model.geo.addPoint(.1, .3, 0, lc, 3)
    gmsh.model.geo.addPoint(0, .3, 0, lc, 4)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(3, 2, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 1, 4)
    gmsh.model.geo.addCurveLoop([4, 1, -2, 3], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.addPhysicalGroup(1, [1, 2, 4], 5)
    gmsh.model.addPhysicalGroup(2, [1], name="My surface")

    # extrusion along the z axis (keeping the 2D mesh from the XY plane)
    h = 0.1
    ov = gmsh.model.geo.extrude([(2, 1)], # extruding surface (dim 2) with tag 1
                                0, 0, h,  # extrusion vector
                                [8, 2],   # extrude the 2D mesh 2 layers: 8/2 subdivisions
                                [0.5, 1]) # both layers height is h/2



    # revolve: rotational extrusion
    ov = gmsh.model.geo.revolve([(2, 28)],    # rotate surface (dim 2) with tag 28
                                -0.1, 0, 0.1, # centre of rotation
                                0, 1, 0,      # rotation axis Y
                                -np.pi / 2,   # revolve by 90 degrees (only angles < Pi)
                                [7])          # 7 subdivisions


    # twist: rotation + translation
    angle = gmsh.onelab.getNumber('Parameters/Twisting angle')[0]   # ONELAB interactive in GUI
    ov = gmsh.model.geo.twist([(2, 50)],
                              0, 0.15, 0.25,  # rotation centre
                              -2 * h, 0, 0,   # translation
                              1, 0, 0,        # rotational axis
                              angle * np.pi / 180.,  # angle
                              [10],  # 10 subdivisions
                              [],    # ?
                              True)  # recombine or not

    gmsh.model.geo.synchronize()


    # extrusion functions return a vector of extruded entities:
    # "top" of the extruded surface (in `ov[0]'), the created volume (in `ov[1]')
    # and the tags of the side surfaces (in `ov[2]', `ov[3]', ...).

    # grouping all three volumes
    gmsh.model.addPhysicalGroup(3, [1, 2, ov[1][1]], 101)

    gmsh.model.mesh.generate(3)
    gmsh.write(f"./meshes/{Name}.msh")

# options
gmsh.option.setNumber("Geometry.PointNumbers", 1)   # visible point tags
gmsh.option.setColor("Geometry.Color.Points", 255, 165, 0)  # setting the colors
gmsh.option.setColor("General.Color.Text", 255, 255, 255)   # setting the colors
gmsh.option.setColor("Mesh.Color.Points", 255, 0, 0)        # setting the colors

# "Color." can be ommitted in color options:
r, g, b, a = gmsh.option.getColor("Geometry.Points")
gmsh.option.setColor("Geometry.Surfaces", r, g, b, a)

# ONELAB parameters can be modified interactively in the GUI
# defined by JSON-strings https://gitlab.onelab.info/doc/tutorials/-/wikis/ONELAB-JSON-interface
gmsh.onelab.set("""[
  {
    "type":"number",
    "name":"Parameters/Twisting angle",
    "values":[90],
    "min":0,
    "max":120,
    "step":1
  }
]""")

# Create the geometry and mesh it:
#createGeometryAndMesh("test_mesh3")

# Launch the GUI and handle the "check" event (recorded in the "ONELAB/Action"
# parameter) to recreate the geometry with a new twisting angle if necessary:
def checkForEvent():
    action = gmsh.onelab.getString("ONELAB/Action")
    if len(action) and action[0] == "check":
        gmsh.onelab.setString("ONELAB/Action", [""])
        createGeometryAndMesh("test_mesh3")
        gmsh.graphics.draw()
    return True


# launch the GUI
gmsh.fltk.initialize()
while gmsh.fltk.isAvailable() and checkForEvent():
    gmsh.fltk.wait()


gmsh.finalize()


-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.1
Packaged by   : runner
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
Info    : Done
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing cu

In [28]:
# Tutorial 4 - Holes in surfaces, annotations, entity colors


import gmsh
import math
import sys
import os

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()


gmsh.model.add("test_mesh3")

# preparing parameters
cm = 1e-02
e1 = 4.5 * cm
e2 = 6 * cm / 2
e3 = 5 * cm / 2
h1 = 5 * cm
h2 = 10 * cm
h3 = 5 * cm
h4 = 2 * cm
h5 = 4.5 * cm
R1 = 1 * cm
R2 = 1.5 * cm
r = 1 * cm
Lc1 = 0.01     # mesh size 1
Lc2 = 0.003    # mesh size 2

def hypot(a, b):
    return math.sqrt(a * a + b * b)

ccos = (-h5 * R1 + e2 * hypot(h5, hypot(e2, R1))) / (h5 * h5 + e2 * e2)
ssin = math.sqrt(1 - ccos * ccos)


# can redefine a namespace:
gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

# defining points and tagging them
gmg.addPoint(-e1 - e2, 0, 0, Lc1, 1)
gmg.addPoint(-e1 - e2, h1, 0, Lc1, 2)
gmg.addPoint(-e3 - r, h1, 0, Lc2, 3)
gmg.addPoint(-e3 - r, h1 + r, 0, Lc2, 4)
gmg.addPoint(-e3, h1 + r, 0, Lc2, 5)
gmg.addPoint(-e3, h1 + h2, 0, Lc1, 6)
gmg.addPoint(e3, h1 + h2, 0, Lc1, 7)
gmg.addPoint(e3, h1 + r, 0, Lc2, 8)
gmg.addPoint(e3 + r, h1 + r, 0, Lc2, 9)
gmg.addPoint(e3 + r, h1, 0, Lc2, 10)
gmg.addPoint(e1 + e2, h1, 0, Lc1, 11)
gmg.addPoint(e1 + e2, 0, 0, Lc1, 12)
gmg.addPoint(e2, 0, 0, Lc1, 13)

gmg.addPoint(R1 / ssin, h5 + R1 * ccos, 0, Lc2, 14)
gmg.addPoint(0, h5, 0, Lc2, 15)
gmg.addPoint(-R1 / ssin, h5 + R1 * ccos, 0, Lc2, 16)
gmg.addPoint(-e2, 0.0, 0, Lc1, 17)

gmg.addPoint(-R2, h1 + h3, 0, Lc2, 18)
gmg.addPoint(-R2, h1 + h3 + h4, 0, Lc2, 19)
gmg.addPoint(0, h1 + h3 + h4, 0, Lc2, 20)
gmg.addPoint(R2, h1 + h3 + h4, 0, Lc2, 21)
gmg.addPoint(R2, h1 + h3, 0, Lc2, 22)
gmg.addPoint(0, h1 + h3, 0, Lc2, 23)

gmg.addPoint(0, h1 + h3 + h4 + R2, 0, Lc2, 24)
gmg.addPoint(0, h1 + h3 - R2, 0, Lc2, 25)

# adding lines
gmg.addLine(1, 17, 1)
gmg.addLine(17, 16, 2)
gmg.addLine(14, 13, 4)
gmg.addLine(13, 12, 5)
gmg.addLine(12, 11, 6)
gmg.addLine(11, 10, 7)
gmg.addLine(8, 7, 9)
gmg.addLine(7, 6, 10)
gmg.addLine(6, 5, 11)
gmg.addLine(3, 2, 13)
gmg.addLine(2, 1, 14)
gmg.addLine(18, 19, 15)
gmg.addLine(21, 22, 20)

# other curve primitives than straight lines:
# - splines, B-splines, circle arcs, ellipse arcs
gmg.addCircleArc(14,   # start point tag
                 15,   # circle centre tag
                 16,   # end point
                 3)    # new tag
gmg.addCircleArc(8, 9, 10, 8)
gmg.addCircleArc(3, 4, 5, 12)
gmg.addCircleArc(21, 20, 24, 16)
gmg.addCircleArc(24, 20, 19, 17)
gmg.addCircleArc(18, 23, 25, 18)
gmg.addCircleArc(25, 23, 22, 19)


# We can then define a new surface:
gmg.addCurveLoop([17, -15, 18, 19, -20, 16], 21)
gmg.addPlaneSurface([21], 22)

# define the exterior surface
gmg.addCurveLoop([11, -12, 13, 14, 1, 2, -3, 4, 5, 6, 7, -8, 9, 10], 23)
gmg.addPlaneSurface([23, 21], 24)  # surface with 2 loops (23,21): exterior + hole

# synchronize the model
gmg.synchronize()

# adding some comments by creating a post-processing view
v = gmsh.view.add("comments")

# Add text in *window coordinates*  10 pixels from the left and 10 pixels up
gmsh.view.addListDataString(v, [10, -10], ["This is my string about the mesh"])

# Add text in *model coordinates* centered at (X,Y,Z) = (0, 0.11, 0)
gmsh.view.addListDataString(v, [0, 0.11, 0], ["Hole"],
                            ["Align", "Center", "Font", "Helvetica"])

# 3D annotations with images
png = './meshes/t4_image-to-display.png'
gmsh.view.addListDataString(v, [0, 0.09, 0],   # 3D annotations coords
                            ["file://" + png + "@0.01x0"], # size in model coords
                            ["Align", "Center"])

# The 3D orientation of the image (the direction of B and L edge in model space:
gmsh.view.addListDataString(v, [-0.01, 0.09, 0],
                            ["file://" + png + "@0.01x0,0,0,1,0,1,0"])

# The image in "billboard" mode, always parallel to the camera
gmsh.view.addListDataString(v, [0, 0.12, 0], ["file://" + png + "@0.01x0#"],
                            ["Align", "Center"])

# The size of 2D annotations is given directly in pixels:
gmsh.view.addListDataString(v, [150, -100], ["file://" + png + "@200x0"])

# Views responding to double-click events:
gmsh.view.option.setString(v, "DoubleClickedCommand",
                           "Printf('View[0] has been double-clicked!');")

# geometrical entities responding to double-click events
gmsh.option.setString(
    "Geometry.DoubleClickedLineCommand",
    "Printf('Curve %g has been double-clicked!', "
    "Geometry.DoubleClickedEntityTag);")

# change the color of some entities:
gmsh.model.setColor([(2, 22)], 127, 127, 127)  # Gray50
gmsh.model.setColor([(2, 24)], 160, 32, 240)   # Purple
gmsh.model.setColor([(1, i) for i in range(1, 15)], 255, 0, 0)     # Red
gmsh.model.setColor([(1, i) for i in range(15, 21)], 255, 255, 0)  # Yellow

gmsh.model.mesh.generate(2)

gmsh.write("test_mesh4.msh")

# Launch the GUI
gmsh.fltk.run()

# finish the session
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 30%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Circle)
Info    : [ 40%] Meshing curve 9 (Line)
Info    : [ 50%] Meshing curve 10 (Line)
Info    : [ 50%] Meshing curve 11 (Line)
Info    : [ 60%] Meshing curve 12 (Circle)
Info    : [ 60%] Meshing curve 13 (Line)
Info    : [ 70%] Meshing curve 14 (Line)
Info    : [ 70%] Meshing curve 15 (Line)
Info    : [ 80%] Meshing curve 16 (Circle)
Info    : [ 80%] Meshing curve 17 (Circle)
Info    : [ 90%] Meshing curve 18 (Circle)
Info    : [ 90%] Meshing curve 19 (Circle)
Info    : [100%] Meshing curve 20 (Line)
Info    : Done meshing 1D (Wall 0.010267s, CPU 0.01044s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 22 (Plane, Frontal-Delaunay)
Info    

In [29]:
# Tutorial 5 - Mesh sizes, holes in volumes

import gmsh
import math
import sys

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh5"
gmsh.model.add(Name)


# target mesh sizes
lc1 = .1
lc2 = .0005
lc3 = .055

# namespace for gmsh.model.geo
gmg = gmsh.model.geo

# defining a truncated cube:
gmg.addPoint(0.5, 0.5, 0.5, lc2, 1)
gmg.addPoint(0.5, 0.5, 0, lc1, 2)
gmg.addPoint(0, 0.5, 0.5, lc1, 3)
gmg.addPoint(0, 0, 0.5, lc1, 4)
gmg.addPoint(0.5, 0, 0.5, lc1, 5)
gmg.addPoint(0.5, 0, 0, lc1, 6)
gmg.addPoint(0, 0.5, 0, lc1, 7)
gmg.addPoint(0, 1, 0, lc1, 8)
gmg.addPoint(1, 1, 0, lc1, 9)
gmg.addPoint(0, 0, 1, lc1, 10)
gmg.addPoint(0, 1, 1, lc1, 11)
gmg.addPoint(1, 1, 1, lc1, 12)
gmg.addPoint(1, 0, 1, lc1, 13)
gmg.addPoint(1, 0, 0, lc1, 14)

# adding the lines
gmg.addLine(8, 9, 1)
gmg.addLine(9, 12, 2)
gmg.addLine(12, 11, 3)
gmg.addLine(11, 8, 4)
gmg.addLine(9, 14, 5)
gmg.addLine(14, 13, 6)
gmg.addLine(13, 12, 7)
gmg.addLine(11, 10, 8)
gmg.addLine(10, 13, 9)
gmg.addLine(10, 4, 10)
gmg.addLine(4, 5, 11)
gmg.addLine(5, 6, 12)
gmg.addLine(6, 2, 13)
gmg.addLine(2, 1, 14)
gmg.addLine(1, 3, 15)
gmg.addLine(3, 7, 16)
gmg.addLine(7, 2, 17)
gmg.addLine(3, 4, 18)
gmg.addLine(5, 1, 19)
gmg.addLine(7, 8, 20)
gmg.addLine(6, 14, 21)

# adding loops and surfaces
gmg.addCurveLoop([-11, -19, -15, -18], 22)
gmg.addPlaneSurface([22], 23)
gmg.addCurveLoop([16, 17, 14, 15], 24)
gmg.addPlaneSurface([24], 25)
gmg.addCurveLoop([-17, 20, 1, 5, -21, 13], 26)
gmg.addPlaneSurface([26], 27)
gmg.addCurveLoop([-4, -1, -2, -3], 28)
gmg.addPlaneSurface([28], 29)
gmg.addCurveLoop([-7, 2, -5, -6], 30)
gmg.addPlaneSurface([30], 31)
gmg.addCurveLoop([6, -9, 10, 11, 12, 21], 32)
gmg.addPlaneSurface([32], 33)
gmg.addCurveLoop([7, 3, 8, 9], 34)
gmg.addPlaneSurface([34], 35)
gmg.addCurveLoop([-10, 18, -16, -20, 4, -8], 36)
gmg.addPlaneSurface([36], 37)
gmg.addCurveLoop([-14, -13, -12, 19], 38)
gmg.addPlaneSurface([38], 39)

shells = []
sl = gmg.addSurfaceLoop([35, 31, 29, 37, 33, 23, 39, 25, 27])
shells.append(sl)

# This function will create a spherical hole in a volume
def cheeseHole(x, y, z, r, lc, shells):

    p1 = gmsh.model.geo.addPoint(x, y, z, lc)
    p2 = gmsh.model.geo.addPoint(x + r, y, z, lc)
    p3 = gmsh.model.geo.addPoint(x, y + r, z, lc)
    p4 = gmsh.model.geo.addPoint(x, y, z + r, lc)
    p5 = gmsh.model.geo.addPoint(x - r, y, z, lc)
    p6 = gmsh.model.geo.addPoint(x, y - r, z, lc)
    p7 = gmsh.model.geo.addPoint(x, y, z - r, lc)

    c1 = gmsh.model.geo.addCircleArc(p2, p1, p7)    # adding 1/4 arcs, 12x
    c2 = gmsh.model.geo.addCircleArc(p7, p1, p5)
    c3 = gmsh.model.geo.addCircleArc(p5, p1, p4)
    c4 = gmsh.model.geo.addCircleArc(p4, p1, p2)
    c5 = gmsh.model.geo.addCircleArc(p2, p1, p3)
    c6 = gmsh.model.geo.addCircleArc(p3, p1, p5)
    c7 = gmsh.model.geo.addCircleArc(p5, p1, p6)
    c8 = gmsh.model.geo.addCircleArc(p6, p1, p2)
    c9 = gmsh.model.geo.addCircleArc(p7, p1, p3)
    c10 = gmsh.model.geo.addCircleArc(p3, p1, p4)
    c11 = gmsh.model.geo.addCircleArc(p4, p1, p6)
    c12 = gmsh.model.geo.addCircleArc(p6, p1, p7)

    l1 = gmsh.model.geo.addCurveLoop([c5, c10, c4])  # adding 1/8 of the sphere, 8x
    l2 = gmsh.model.geo.addCurveLoop([c9, -c5, c1])
    l3 = gmsh.model.geo.addCurveLoop([c12, -c8, -c1])
    l4 = gmsh.model.geo.addCurveLoop([c8, -c4, c11])
    l5 = gmsh.model.geo.addCurveLoop([-c10, c6, c3])
    l6 = gmsh.model.geo.addCurveLoop([-c11, -c3, c7])
    l7 = gmsh.model.geo.addCurveLoop([-c2, -c7, -c12])
    l8 = gmsh.model.geo.addCurveLoop([-c6, -c9, c2])

    # function addSurfaceFilling()' for non-plane surfaces
    s1 = gmsh.model.geo.addSurfaceFilling([l1])     # 1/8 of sphere 8 surfaces
    s2 = gmsh.model.geo.addSurfaceFilling([l2])
    s3 = gmsh.model.geo.addSurfaceFilling([l3])
    s4 = gmsh.model.geo.addSurfaceFilling([l4])
    s5 = gmsh.model.geo.addSurfaceFilling([l5])
    s6 = gmsh.model.geo.addSurfaceFilling([l6])
    s7 = gmsh.model.geo.addSurfaceFilling([l7])
    s8 = gmsh.model.geo.addSurfaceFilling([l8])

    sl = gmsh.model.geo.addSurfaceLoop([s1, s2, s3, s4, s5, s6, s7, s8])
    v = gmsh.model.geo.addVolume([sl])
    shells.append(sl)
    return v


# We create five holes in the cube:
x = 0
y = 0.75
z = 0
r = 0.09
for t in range(1, 6):
    x += 0.166
    z += 0.166
    v = cheeseHole(x, y, z, r, lc3, shells)
    gmg.addPhysicalGroup(3, [v], t)

# Volume of the cube with holes is defined by 6 surface loops: exterior + 5 holes
gmg.addVolume(shells, 186)

gmg.synchronize()


# physical volume for the hollowed cube
gmsh.model.addPhysicalGroup(3, [186], 10)

#   to only mesh a subset:
# ent = gmsh.model.getEntities()
# gmsh.model.setVisibility(ent, False)
# gmsh.model.setVisibility([(3, 5)], True, True)
# gmsh.option.setNumber("Mesh.MeshOnlyVisible", 1)

# Meshing algorithms can changed globally using options:
gmsh.option.setNumber("Mesh.Algorithm", 6)  # Frontal-Delaunay for 2D meshes
#
# changing mesh algs for individual surfaces
gmsh.model.mesh.setAlgorithm(2, 33, 1)  # settings on surface 1
#
# Optimization: 'Geometric validity of curvilinear finite elements' Journal Comp.Physics
# gmsh.option.setNumber("Mesh.ElementOrder", 2)
# gmsh.option.setNumber("Mesh.HighOrderOptimize", 2)

gmg.synchronize()###
gmsh.model.mesh.generate(3)
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 20%] Meshing curve 10 (Line)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 30%] Meshing curve 18 (Line)
Info    : [ 30%] Meshing curve 19 (Line)
Info    : [ 30%] Meshing curve 20 (Line)
Info    : [ 30%] Meshing curve 21 (Line)
Info    : [ 30%] Meshing curve 22 (Circle)
Info    : [ 30%] Meshing curve 23 (Circle)
Info    : [ 30%] Meshing curve 24 (Cir

In [120]:
# Tutorial 6 - Transfinite meshes, deleting entities

import gmsh
import math
import sys

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh6"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# Copied from tutorial 1
lc = 1e-2
gmg.addPoint(0, 0, 0, lc, 1)
gmg.addPoint(.1, 0, 0, lc, 2)
gmg.addPoint(.1, .3, 0, lc, 3)
gmg.addPoint(0, .3, 0, lc, 4)
gmg.addLine(1, 2, 1)
gmg.addLine(3, 2, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 1, 4)
gmg.addCurveLoop([4, 1, -2, 3], 1)
gmg.addPlaneSurface([1], 1)

# Delete the surface and the left line, and replace the line with 3 new ones:
gmg.remove([(2, 1)])  # delete type 2 (surface), tag 1
gmg.remove([(1, 4)])  # delete type 1 (line), tag 4
gmg.remove([(2, 1), (1,4)])  # delete tag 1 *and* tag 4


p1 = gmg.addPoint(-0.05, 0.05, 0, lc)
p2 = gmg.addPoint(-0.05, 0.1, 0, lc)
l1 = gmg.addLine(1, p1)
l2 = gmg.addLine(p1, p2)
l3 = gmg.addLine(p2, 4)

# Create surface
gmg.addCurveLoop([2, -1, l1, l2, l3, -3], 2)
gmg.addPlaneSurface([-2], 1)


# ftion 'setTransfiniteCurve()' explicitly specifies location of mesh nodes on the curve.
gmg.mesh.setTransfiniteCurve(2, 20)  # 20 uniformly distributed points
gmg.mesh.setTransfiniteCurve(l1, 6)  # 6 uniform points
gmg.mesh.setTransfiniteCurve(l2, 6)
gmg.mesh.setTransfiniteCurve(l3, 10)

# geometric series location of the mesh
gmg.mesh.setTransfiniteCurve(1, 30, "Progression", -1.2)  # geom. distribution (reversed)
gmg.mesh.setTransfiniteCurve(3, 30, "Progression", 1.2)   # geom. distribution

# meshing the surface
gmg.mesh.setTransfiniteSurface(1, "Left", [1, 2, 3, 4])  # set up the corners of interpolation

# To use quadrilateral mesh elements:
gmg.mesh.setRecombine(2, 1)

# more points
gmg.addPoint(0.2, 0.2, 0, 1.0, 7)
gmg.addPoint(0.2, 0.1, 0, 1.0, 8)
gmg.addPoint(0, 0.3, 0, 1.0, 9)
gmg.addPoint(0.25, 0.2, 0, 1.0, 10)
gmg.addPoint(0.3, 0.1, 0, 1.0, 11)
gmg.addLine(8, 11, 10)
gmg.addLine(11, 10, 11)
gmg.addLine(10, 7, 12)
gmg.addLine(7, 8, 13)
gmg.addCurveLoop([13, 10, 11, 12], 14)
gmg.addPlaneSurface([14], 15)
for i in range(10, 14):
    gmg.mesh.setTransfiniteCurve(i, 10)
#gmg.mesh.setTransfiniteSurface(15)  # no need for corner list if mesh <= 4 corners
gmg.mesh.setTransfiniteSurface(15,
                              "Alternate")  # optional Left/Right/Alternate mesh

gmg.synchronize()

# apply elliptic smoother to the mesh grid:
gmsh.option.setNumber("Mesh.Smoothing", 100)

gmsh.model.mesh.generate(2)
# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 10 (Line)
Info    : [ 70%] Meshing curve 11 (Line)
Info    : [ 80%] Meshing curve 12 (Line)
Info    : [ 90%] Meshing curve 13 (Line)
Info    : Done meshing 1D (Wall 0.00101562s, CPU 0.001396s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 50%] Meshing surface 15 (Transfinite)
Info    : Done meshing 2D (Wall 0.00132657s, CPU 0.00134s)
Info    : 701 nodes 856 elements
Info    : Writing './meshes/test_mesh6.msh'...
Info    : Done writing './meshes/test_mesh6.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750

In [53]:
# Tutorial 7 - Background meshes

import gmsh
import math
import sys

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

# Merge a list-based post-processing view with the mesh sizes (2D function):
gmsh.merge("./meshes/background_mesh.pos")

# add model
Name = "test_mesh7"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# Create a simple rectangular geometry:
lc = 1e-2
gmg.addPoint(0, 0, 0, lc, 1)
gmg.addPoint(.1, 0, 0, lc, 2)
gmg.addPoint(.1, .3, 0, lc, 3)
gmg.addPoint(0, .3, 0, lc, 4)
gmg.addLine(1, 2, 1)
gmg.addLine(3, 2, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 1, 4)
gmg.addCurveLoop([4, 1, -2, 3], 1)
gmg.addPlaneSurface([1], 1)
gmg.synchronize()


# Add the post-processing view as a new size field:
bg_field = gmsh.model.mesh.field.add("PostView")
gmsh.model.mesh.field.setNumber(bg_field, "ViewIndex", 0)

# Apply the view as the current background mesh size field:
gmsh.model.mesh.field.setAsBackgroundMesh(bg_field)

# Computing the mesh sizes from the background mesh only (do not use lc params)
gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
gmg.synchronize()


# generate the mesh
gmsh.model.mesh.generate(2)

# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Reading './meshes/background_mesh.pos'...
Info    : Done reading './meshes/background_mesh.pos'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.033759s, CPU 0.033168s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0669236s, CPU 0.06657s)
Info    : 1788 nodes 3578 elements
Info    : Writing './meshes/test_mesh7.msh'...
Info    : Done writing './meshes/test_mesh7.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh

In [1]:
# Tutorial 8 - Post-processing, image export and animations
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh8"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# define geometry
lc = 1e-2
gmg.addPoint(0, 0, 0, lc, 1)
gmg.addPoint(.1, 0, 0, lc, 2)
gmg.addPoint(.1, .3, 0, lc, 3)
gmg.addPoint(0, .3, 0, lc, 4)
gmg.addLine(1, 2, 1)
gmg.addLine(3, 2, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 1, 4)
gmg.addCurveLoop([4, 1, -2, 3], 1)
gmg.addPlaneSurface([1], 1)

gmsh.model.geo.synchronize()

# We merge some post-processing views to work on
gmsh.merge('./meshes/view1.pos')
gmsh.merge('./meshes/view1.pos')
gmsh.merge('./meshes/view4.pos')  # contains 2 views inside

# We then set some general options:
gmsh.option.setNumber("General.Trackball", 0)
gmsh.option.setNumber("General.RotationX", 0)
gmsh.option.setNumber("General.RotationY", 0)
gmsh.option.setNumber("General.RotationZ", 0)

white = (255, 255, 255)
black = (0, 0, 0)

# Color options are special
# Setting a color option of "X.Y" actually sets the option "X.Color.Y"
# Sets "General.Color.Background", etc.
gmsh.option.setColor("General.Background", white[0], white[1], white[2])
gmsh.option.setColor("General.Foreground", black[0], black[1], black[2])
gmsh.option.setColor("General.Text", black[0], black[1], black[2])

gmsh.option.setNumber("General.Orthographic", 0) # perspective/orthographic
gmsh.option.setNumber("General.Axes", 1)         # print axis 0/1
gmsh.option.setNumber("General.SmallAxes", 0)

# We also set some options for each post-processing view:

# checking the num of views
v = gmsh.view.getTags()
if len(v) != 4:
    gmsh.logger.write(f"Wrong number of views!{len(v)}", "error")
    gmsh.finalize()
    exit()

# We set some options for each post-processing view:
gmsh.view.option.setNumber(v[0], "IntervalsType", 2)
gmsh.view.option.setNumber(v[0], "OffsetZ", 0.05)
gmsh.view.option.setNumber(v[0], "RaiseZ", 0)
gmsh.view.option.setNumber(v[0], "Light", 1)
gmsh.view.option.setNumber(v[0], "ShowScale", 0)
gmsh.view.option.setNumber(v[0], "SmoothNormals", 1)

gmsh.view.option.setNumber(v[1], "IntervalsType", 1)
# Note that we can't yet set the ColorTable in API
gmsh.view.option.setNumber(v[1], "NbIso", 10)
gmsh.view.option.setNumber(v[1], "ShowScale", 0)

gmsh.view.option.setString(v[2], "Name", "Test plotting")
gmsh.view.option.setNumber(v[2], "Axes", 1)
gmsh.view.option.setNumber(v[2], "IntervalsType", 2)
gmsh.view.option.setNumber(v[2], "Type", 2)
gmsh.view.option.setNumber(v[2], "IntervalsType", 2)
gmsh.view.option.setNumber(v[2], "AutoPosition", 0)
gmsh.view.option.setNumber(v[2], "PositionX", 85)
gmsh.view.option.setNumber(v[2], "PositionY", 50)
gmsh.view.option.setNumber(v[2], "Width", 200)
gmsh.view.option.setNumber(v[2], "Height", 130)
gmsh.view.option.setNumber(v[3], "Visible", 0)




# creating an animation
t = 0  # Initial step

for num in range(1, 4):

    # Set time step
    for vv in v:
        gmsh.view.option.setNumber(vv, "TimeStep", t)
    current_step = gmsh.view.option.getNumber(v[0], "TimeStep")
    max_step = gmsh.view.option.getNumber(v[0], "NbTimeStep") - 1
    if current_step < max_step:
        t = t + 1
    else:
        t = 0

    gmsh.view.option.setNumber(v[0], "RaiseZ",
                               gmsh.view.option.getNumber(v[0], "RaiseZ") +
                               0.01 / gmsh.view.option.getNumber(v[0], "Max") * t)

    if num == 3:
        # Resize the graphics when num == 3, to create 640x480 frames
        gmsh.option.setNumber("General.GraphicsWidth",
                              gmsh.option.getNumber("General.MenuWidth") + 640)
        gmsh.option.setNumber("General.GraphicsHeight", 480)

    frames = 50
    for num2 in range(frames):
        # Incrementally rotate the scene
        gmsh.option.setNumber("General.RotationX",
                              gmsh.option.getNumber("General.RotationX") + 10)
        gmsh.option.setNumber("General.RotationY",
                              gmsh.option.getNumber("General.RotationX") / 3)
        gmsh.option.setNumber("General.RotationZ",
                              gmsh.option.getNumber("General.RotationZ") + 0.1)

        # Draw the scene
        gmsh.graphics.draw()

        if num == 3:
            # Uncomment the following lines to save each frame to an image file
            #gmsh.write("t8-{}.gif".format(num2))
            #gmsh.write("t8-{}.ppm".format(num2))
            #gmsh.write("t8-{}.jpg".format(num2))
            pass

    if num == 3:
        # Here we could make a system call to generate a movie with ffmeg:
        # import subprocess
        # subprocess.call("ffmpeg -i t8-%d.jpg t8.mpg".split(' '))
        pass



# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Reading './meshes/view1.pos'...
Info    : Done reading './meshes/view1.pos'
Info    : Reading './meshes/view1.pos'...
Info    : Done reading './meshes/view1.pos'
Info    : Reading './meshes/view4.pos'...
Info    : Done reading './meshes/view4.pos'
Info    : Writing './meshes/test_mesh8.msh'...
Info    : Done writing './meshes/test_mesh8.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.1
Packaged by  

In [43]:
# Tutorial 9 - Plugins
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh9"
gmsh.model.add(Name)

# loading a view
gmsh.merge('./meshes/view3.pos')
v = gmsh.view.getTags()
if len(v) != 1:
    gmsh.logger.write("Wrong number of views!", "error")
    gmsh.finalize()
    exit()

# Isosurface' plugin
gmsh.plugin.setNumber("Isosurface", "Value", 0.67)
gmsh.plugin.setNumber("Isosurface", "View", 0)
v1 = gmsh.plugin.run("Isosurface")


# `CutPlane' plugin (plane A*x+B*y+C*z+D=0)
gmsh.plugin.setNumber("CutPlane", "A", 0)
gmsh.plugin.setNumber("CutPlane", "B", 0.2)
gmsh.plugin.setNumber("CutPlane", "C", 1)
gmsh.plugin.setNumber("CutPlane", "D", 0)
gmsh.plugin.setNumber("CutPlane", "View", 0)
v2 = gmsh.plugin.run("CutPlane")

# Add a title (window coords greater than 99999 represents the center)
gmsh.plugin.setString("Annotate", "Text", "A nice little title")
gmsh.plugin.setNumber("Annotate", "X", 1.e5)
gmsh.plugin.setNumber("Annotate", "Y", 50)
gmsh.plugin.setString("Annotate", "Font", "Times-BoldItalic")
gmsh.plugin.setNumber("Annotate", "FontSize", 28)
gmsh.plugin.setString("Annotate", "Align", "Center")
gmsh.plugin.setNumber("Annotate", "View", 0)
gmsh.plugin.run("Annotate")

gmsh.plugin.setString("Annotate", "Text", "(and another little subtitle)")
gmsh.plugin.setNumber("Annotate", "Y", 70)
gmsh.plugin.setString("Annotate", "Font", "Times-Roman")
gmsh.plugin.setNumber("Annotate", "FontSize", 12)
gmsh.plugin.run("Annotate")


# setting options:
gmsh.view.option.setNumber(v[0], "Light", 1)
gmsh.view.option.setNumber(v[0], "IntervalsType", 0)
gmsh.view.option.setNumber(v[0], "NbIso", 10)
gmsh.view.option.setNumber(v[0], "SmoothNormals", 1)
gmsh.view.option.setNumber(v1, "IntervalsType", 2)
gmsh.view.option.setNumber(v2, "IntervalsType", 2)


# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Reading './meshes/view3.pos'...
Info    : Done reading './meshes/view3.pos'
Info    : Running Plugin(Isosurface)...
Info    : Done running Plugin(Isosurface)
Info    : Running Plugin(CutPlane)...
Info    : Done running Plugin(CutPlane)
Info    : Running Plugin(Annotate)...
Info    : Done running Plugin(Annotate)
Info    : Running Plugin(Annotate)...
Info    : Done running Plugin(Annotate)
Info    : Writing './meshes/test_mesh9.msh'...
Info    : Done writing './meshes/test_mesh9.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Pn

In [15]:
# Tutorial 10 - Mesh size fields
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh10"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# adding geometry
# Let's create a simple rectangular geometry:
lc = .15
gmg.addPoint(0.0, 0.0, 0, lc, 1)
gmg.addPoint(1, 0.0, 0, lc, 2)
gmg.addPoint(1, 1, 0, lc, 3)
gmg.addPoint(0, 1, 0, lc, 4)
gmg.addPoint(0.2, .5, 0, lc, 5)

gmg.addLine(1, 2, 1)
gmg.addLine(2, 3, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 1, 4)

gmg.addCurveLoop([1, 2, 3, 4], 5)
gmg.addPlaneSurface([5], 6)
gmg.synchronize()


# we want mesh elements with size lc/30 near curve 2 and point 5, and lc elsewhere
# To achieve this, we use mesh size fields: "Distance" and "Threshold"
gmsh.model.mesh.field.add("Distance", 1)  # create a distance field
gmsh.model.mesh.field.setNumbers(1, "PointsList", [5]) # for point 5
gmsh.model.mesh.field.setNumbers(1, "CurvesList", [2]) # and curve 2
gmsh.model.mesh.field.setNumber(1, "Sampling", 100)    # sample curve 2 at 100 points

# We then define a `Threshold' field, which uses the return value of the
# `Distance' field 1 in order to define a simple change in element size
# SizeMax -                     /------------------
#                              /
#                             /
#                            /
# SizeMin -o----------------/
#          |                |    |
#        Point         DistMin  DistMax
gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "InField", 1)
gmsh.model.mesh.field.setNumber(2, "SizeMin", lc / 30)
gmsh.model.mesh.field.setNumber(2, "SizeMax", lc)
gmsh.model.mesh.field.setNumber(2, "DistMin", 0.15)
gmsh.model.mesh.field.setNumber(2, "DistMax", 0.5)


# Say we want to modulate the mesh element sizes using a mathematical function
# of the spatial coordinates. We can do this with the MathEval field:
gmsh.model.mesh.field.add("MathEval", 3)
gmsh.model.mesh.field.setString(3, "F",
                                "Cos(4*3.14*x) * Sin(4*3.14*y) / 10 + 0.101")

# `Distance' field around point 1 (combine with MathEval field)
gmsh.model.mesh.field.add("Distance", 4)
gmsh.model.mesh.field.setNumbers(4, "PointsList", [1])

# create a 'MathEval' field 5 depends on field 4 (cubic distance to point 1)
gmsh.model.mesh.field.add("MathEval", 5)
gmsh.model.mesh.field.setString(5, "F", "F4^3 + " + str(lc / 100))

# use a `Box' field to impose a step change in element sizes in the box
gmsh.model.mesh.field.add("Box", 6)
gmsh.model.mesh.field.setNumber(6, "VIn", lc / 15)
gmsh.model.mesh.field.setNumber(6, "VOut", lc)
gmsh.model.mesh.field.setNumber(6, "XMin", 0.3)
gmsh.model.mesh.field.setNumber(6, "XMax", 0.6)
gmsh.model.mesh.field.setNumber(6, "YMin", 0.3)
gmsh.model.mesh.field.setNumber(6, "YMax", 0.6)
gmsh.model.mesh.field.setNumber(6, "Thickness", 0.3)

# Let's use the minimum of all the fields as the mesh size field:
gmsh.model.mesh.field.add("Min", 7)   # minimum of all fields
gmsh.model.mesh.field.setNumbers(7, "FieldsList", [2, 3, 5, 6])
gmsh.model.mesh.field.setAsBackgroundMesh(7)

# The API allows to set a global mesh size callback
def meshSizeCallback(dim, tag, x, y, z, lc):
    return min(lc, 0.02 * x + 0.01)
gmsh.model.mesh.setSizeCallback(meshSizeCallback)


# To determine the size of mesh elements, Gmsh locally computes the minimum of:
#   1) the size of the model bounding box;
#   2) if `Mesh.MeshSizeFromPoints' is set, the mesh size specified at geometrical
#      points;
#   3) if `Mesh.MeshSizeFromCurvature' is positive, the mesh size based on
#      curvature (the value specifying the number of elements per 2 * pi rad);
#   4) the background mesh size field;
#   5) any per-entity mesh size constraint;


# When the element size is fully specified by a mesh size field, we can set:
gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
gmsh.model.setColor([(2, 6)], 255, 0, 0)  # Red
gmg.synchronize()

# "Frontal-Delaunay" (6) 2D meshing algorithm -> highest quality meshes
# "Delaunay" (5) 2D meshing algorithm -> handles complex mesh size fields better
gmsh.option.setNumber("Mesh.Algorithm", 5)  # setting the meshing algorithm

gmsh.model.mesh.generate(2)

# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.136817s, CPU 0.136003s)
Info    : Meshing 2D...
Info    : Meshing surface 6 (Plane, Delaunay)
Info    : Done meshing 2D (Wall 1.04521s, CPU 1.03404s)
Info    : 25111 nodes 50223 elements
Info    : Writing './meshes/test_mesh10.msh'...
Info    : Done writing './meshes/test_mesh10.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugi

In [23]:
# Tutorial 11 - Unstructured quadrangular meshes
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh11"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# Unstructured meshes recombined into quadrilateral (quadrangular) mesh
p1 = gmg.addPoint(-1.25, -.5, 0)
p2 = gmg.addPoint(1.25, -.5, 0)
p3 = gmg.addPoint(1.25, 1.25, 0)
p4 = gmg.addPoint(-1.25, 1.25, 0)

l1 = gmg.addLine(p1, p2)
l2 = gmg.addLine(p2, p3)
l3 = gmg.addLine(p3, p4)
l4 = gmg.addLine(p4, p1)

cl = gmg.addCurveLoop([l1, l2, l3, l4])
pl = gmg.addPlaneSurface([cl])
gmg.synchronize()

# adding a mesh field
field = gmsh.model.mesh.field
field.add("MathEval", 1)
field.setString(1, "F", "0.01*(1.0+30.*(y-x*x)*(y-x*x) + (1-x)*(1-x))")
field.setAsBackgroundMesh(1)

# To generate quadrangles instead of triangles, we use:
gmsh.model.mesh.setRecombine(2, pl)
# gmsh.option.setNumber("Mesh.Algorithm", 8)  # using FrontalDelaney for quads (optional)

# if there are some triangles left - 2 methods:
# gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 2) # coarses recombination (2 or 3)
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)   # subdivision step alone

# generate the mesh
gmsh.model.mesh.generate(2)
gmsh.model.setColor([(2, pl)], 255, 0, 0)  # Red

# if we want to recombine the mesh after generating it, we use mesh.refine()
# gmsh.model.mesh.generate(2)
# gmsh.model.mesh.recombine()
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
# gmsh.model.mesh.refine()


# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.0198121s, CPU 0.019568s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Blossom: 10752 internal 66 closed
Info    : Blossom recombination completed (Wall 0.16523s, CPU 0.162745s): 3485 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.777912, min Q = 0.341789
Info    : Done meshing 2D (Wall 0.316135s, CPU 0.311934s)
Info    : 3519 nodes 3555 elements
Info    : Writing './meshes/test_mesh11.msh'...
Info    : Done writing './meshes/test_mesh11.msh'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg B

In [34]:
# Tutorial 12 - Cross-patch meshing with compounds
import gmsh

# "Compound" meshing constraints allow to generate meshes across surface
# boundaries, which can be useful e.g. for imported CAD models (e.g. STEP) with
# undesired small features "we want to simplify the mesh at surface boundaries"

# When a `setCompound()' meshing constraint is given, at mesh generation time
# Gmsh
#  1. meshes the underlying elementary geometrical entities, individually
#  2. creates a discrete entity that combines all the individual meshes
#  3. computes a discrete parametrization (i.e. a piece-wise linear mapping)
#     on this discrete entity
#  4. meshes the discrete entity using this discrete parametrization instead
#     of the underlying geometrical description of the underlying elementary
#     entities making up the compound
#  5. optionally, reclassifies the mesh elements and nodes on the original
#     entities


# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh12"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

lc = 0.1
gmg.addPoint(0, 0, 0, lc, 1)
gmg.addPoint(1, 0, 0, lc, 2)
gmg.addPoint(1, 1, 0.5, lc, 3)
gmg.addPoint(0, 1, 0.4, lc, 4)
gmg.addPoint(0.3, 0.2, 0, lc, 5)
gmg.addPoint(0, 0.01, 0.01, lc, 6)
gmg.addPoint(0, 0.02, 0.02, lc, 7)
gmg.addPoint(1, 0.05, 0.02, lc, 8)
gmg.addPoint(1, 0.32, 0.02, lc, 9)

gmg.addLine(1, 2, 1)
gmg.addLine(2, 8, 2)
gmg.addLine(8, 9, 3)
gmg.addLine(9, 3, 4)
gmg.addLine(3, 4, 5)
gmg.addLine(4, 7, 6)
gmg.addLine(7, 6, 7)
gmg.addLine(6, 1, 8)
gmg.addSpline([7, 5, 9], 9)
gmg.addLine(6, 8, 10)

gmg.addCurveLoop([5, 6, 9, 4], 11)
gmg.addSurfaceFilling([11], 1)
gmg.addCurveLoop([-9, 3, 10, 7], 13)
gmg.addSurfaceFilling([13], 5)
gmg.addCurveLoop([-10, 2, 1, 8], 15)
gmg.addSurfaceFilling([15], 10)
gmg.synchronize()

# Treat curves 2, 3 and 4 as a single curve when meshing
gmsh.model.mesh.setCompound(1, [2, 3, 4])
# same with curves 6, 7 and 8
gmsh.model.mesh.setCompound(1, [6, 7, 8])
# treat surfaces 1, 5 and 10 as a single surface when meshing
gmsh.model.mesh.setCompound(2, [1, 5, 10])

# generate
gmsh.model.mesh.generate(2)


# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 70%] Meshing curve 8 (Line)
Info    : [ 80%] Meshing curve 9 (Nurb)
Info    : [ 90%] Meshing curve 10 (Line)
Info    : [100%] Meshing curve 2 (Line)
Info    : [100%] Meshing curve 100002 (Discrete curve)
Info    : [100%] Meshing curve 6 (Line)
Info    : [100%] Meshing curve 100006 (Discrete curve)
Info    : Done meshing 1D (Wall 0.00465463s, CPU 0.005008s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Surface, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 5 (Surface, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 10 (Surface, Frontal-Delaunay)
Info    : [100%] Meshing surface 1 (Surface, Frontal-Delaunay)
Info    : [100%] Meshing surface 100001 (Discret

In [2]:
# Tutorial 13 - Remeshing an STL file without an underlying CAD model
import gmsh
import numpy as np
import os

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh13"
gmsh.model.add(Name)

def createGeometryAndMesh(Name):
    # Clear all models and merge an STL mesh that we would like to remesh (from
    # the parent directory):
    gmsh.clear()
    gmsh.merge('./meshes/t13_data.stl')

    # splitting the original surface along sharp geometrical features

    # Angle above which an edge is considered as sharp: interactively from oneLab
    angle = gmsh.onelab.getNumber('Parameters/Angle for surface detection')[0]

    # For complex geometries force the creation of simpler patches:
    forceParametrizablePatches = gmsh.onelab.getNumber(
        'Parameters/Create surfaces guaranteed to be parametrizable')[0]

    # For open surfaces include the boundary edges in the class. process
    includeBoundary = True

    # Force curves to be split on given angle:
    curveAngle = 180

    gmsh.model.mesh.classifySurfaces(angle * np.pi / 180., includeBoundary,
                                     forceParametrizablePatches,
                                     curveAngle * np.pi / 180.)

    # Create a geometry for all the discrete curves and surfaces in the mesh, by
    # computing a parametrization for each one
    gmsh.model.mesh.createGeometry()

    # Create a volume from all the surfaces
    s = gmsh.model.getEntities(2)
    l = gmsh.model.geo.addSurfaceLoop([e[1] for e in s])
    gmsh.model.geo.addVolume([l])

    gmsh.model.geo.synchronize()

    # We specify element sizes imposed by a size field (optional)
    f = gmsh.model.mesh.field.add("MathEval")
    gmsh.model.mesh.field.setString(f, "F", "2*Sin((x+y)/5) + 3")
    gmsh.model.mesh.field.setString(f, "F", "4")  # simple size field
    gmsh.model.mesh.field.setAsBackgroundMesh(f)

    gmsh.model.mesh.generate(3)
    gmsh.write(f"./meshes/{Name}.msh")


# Create ONELAB parameters with remeshing options:
gmsh.onelab.set("""[
  {
    "type":"number",
    "name":"Parameters/Angle for surface detection",
    "values":[40],
    "min":20,
    "max":120,
    "step":1
  },
  {
    "type":"number",
    "name":"Parameters/Create surfaces guaranteed to be parametrizable",
    "values":[0],
    "choices":[0, 1]
  }
]""")

# Create the geometry and mesh it:
createGeometryAndMesh(Name)

# Launch the GUI and handle the "check" event
def checkForEvent():
    action = gmsh.onelab.getString("ONELAB/Action")
    if len(action) and action[0] == "check":
        gmsh.onelab.setString("ONELAB/Action", [""])
        createGeometryAndMesh(Name)
        gmsh.graphics.draw()
    return True

# start the GUI
gmsh.fltk.initialize()
while gmsh.fltk.isAvailable() and checkForEvent():
    gmsh.fltk.wait()

# finish the session
gmsh.finalize()

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reading './meshes/t13_data.stl'...
Info    : 1580 facets in solid 0 Created by Gmsh
Info    : Done reading './meshes/t13_data.stl'
Info    : Classifying surfaces (angle: 40)...
Info    : Found 11 model surfaces
Info    : Found 12 model curves
Info    : Done classifying surfaces (Wall 0.0102819s, CPU 0.010074s)
Info    : Creating geometry of discrete curves...
Info    : Done creating geometry of discrete curves (Wall 4.0885e-05s, CPU 5e-05s)
Info    : Creating geometry of discrete surfaces...
Info    : [ 20%] Discrete surface 4 is planar, simplifying parametrization  
Info    : [ 70%] Discrete surface 9 is planar, simplifying parametrization  
Info    : [ 80%] Discrete surface 10 is planar, simplifying parametrization 
Info    : [ 90%] Discrete surface 11 is planar, simplifying parametrization 
Info    : [100%] Discrete surface 12 is planar, simplifying parametrization 
Info    : Done creat

Info    : Done
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reading './meshes/t13_data.stl'...
Info    : 1580 facets in solid 0 Created by Gmsh
Info    : Done reading './meshes/t13_data.stl'
Info    : Classifying surfaces (angle: 80)...
Info    : Found 10 model surfaces
Info    : Found 11 model curves
Info    : Done classifying surfaces (Wall 0.0106593s, CPU 0.010676s)
Info    : Creating geometry of discrete curves...
Info    : Done creating geometry of discrete curves (Wall 3.4958e-05s, CPU 4.5e-05s)
Info    : Creating geometry of discrete surfaces...
Info    : [ 60%] Discrete surface 8 is planar, simplifying parametrization  
Info    : [ 70%] Discrete surface 9 is planar, simplifying parametrization  
Info    : [ 80%] Discrete surface 10 is planar, simplifying parametrization 
Info    : [ 90%] Discrete surface 11 is planar, simplifying parametrization 
Info    : Done creating geometry of discrete surfaces (Wall 0.00980382s, CPU 0.0

Error   : Invalid exterior boundary mesh for parametrization


Info    : Done
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reading './meshes/t13_data.stl'...
Info    : 1580 facets in solid 0 Created by Gmsh
Info    : Done reading './meshes/t13_data.stl'
Info    : Classifying surfaces (angle: 105)...
Info    : Found 1 model surfaces
Info    : Found 0 model curves
Info    : Done classifying surfaces (Wall 0.0106041s, CPU 0.010564s)
Info    : Creating geometry of discrete surfaces...
Info    : Done creating geometry of discrete surfaces (Wall 0.00220557s, CPU 0.002453s)
Info    : Meshing 1D...
Info    : Done meshing 1D (Wall 6.526e-06s, CPU 6e-06s)
Info    : Meshing 2D...
Info    : Done meshing 2D (Wall 1.63e-05s, CPU 2.3e-05s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 788 nodes...
Info    : Done tetrahedrizing 796 nodes (Wall 0.00800243s, CPU 0.00786s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Id

Error   : Invalid exterior boundary mesh for parametrization


...

Error   : Invalid exterior boundary mesh for parametrization


Info    : Done
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reading './meshes/t13_data.stl'...
Info    : 1580 facets in solid 0 Created by Gmsh
Info    : Done reading './meshes/t13_data.stl'
Info    : Classifying surfaces (angle: 120)...
Info    : Found 1 model surfaces
Info    : Found 0 model curves
Info    : Done classifying surfaces (Wall 0.0105548s, CPU 0.010436s)
Info    : Creating geometry of discrete surfaces...
Info    : Done creating geometry of discrete surfaces (Wall 0.00222093s, CPU 0.002487s)
Info    : Meshing 1D...
Info    : Done meshing 1D (Wall 1.7424e-05s, CPU 2.2e-05s)
Info    : Meshing 2D...
Info    : Done meshing 2D (Wall 1.4743e-05s, CPU 2.4e-05s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 788 nodes...
Info    : Done tetrahedrizing 796 nodes (Wall 0.00767386s, CPU 0.007488s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :

Error   : Invalid exterior boundary mesh for parametrization


Info    : Done
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reading './meshes/t13_data.stl'...
Info    : 1580 facets in solid 0 Created by Gmsh
Info    : Done reading './meshes/t13_data.stl'
Info    : Classifying surfaces (angle: 270)...
Info    : Found 1 model surfaces
Info    : Found 0 model curves
Info    : Done classifying surfaces (Wall 0.00991133s, CPU 0.009829s)
Info    : Creating geometry of discrete surfaces...
Info    : Done creating geometry of discrete surfaces (Wall 0.0025987s, CPU 0.002785s)
Info    : Meshing 1D...
Info    : Done meshing 1D (Wall 5.61299e-06s, CPU 6e-06s)
Info    : Meshing 2D...
Info    : Done meshing 2D (Wall 1.4604e-05s, CPU 2e-05s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 788 nodes...
Info    : Done tetrahedrizing 796 nodes (Wall 0.00791373s, CPU 0.007725s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  -

Error   : Invalid exterior boundary mesh for parametrization


In [3]:
# Tutorial 16 - OpenCASCADE geometry kernel
import gmsh
import sys

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh16"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# We can log all messages for further processing with:
#gmsh.logger.start()

# We first create two cubes:
gmsh.model.occ.addBox(0, 0, 0, 1, 1, 1, 1)
gmsh.model.occ.addBox(0, 0, 0, 0.5, 0.5, 0.5, 2)

# We apply a boolean difference to create the "cube minus one eigth" shape:
gmsh.model.occ.cut([(3, 1)], [(3, 2)], 3)

# Boolean operations with OpenCASCADE always create new entities. By default the
# extra arguments `removeObject' and `removeTool' in `cut()' are set to `True',
# which will delete the original entities.

# We then create the five spheres:
x = 0
y = 0.75
z = 0
r = 0.09
holes = []
for t in range(1, 6):
    x += 0.166
    z += 0.166
    gmsh.model.occ.addSphere(x, y, z, r, 3 + t)
    holes.append((3, 3 + t))

# If we had wanted five empty holes we would have used `cut()' again. Here we
# want five spherical inclusions, whose mesh should be conformal with the mesh
# of the cube: we thus use `fragment()', which intersects all volumes in a
# conformal manner (without creating duplicate interfaces):
ov, ovv = gmsh.model.occ.fragment([(3, 3)], holes)

# ov contains all the generated entities of the same dimension as the input
# entities:
print("fragment produced volumes:")
for e in ov:
    print(e)

# ovv contains the parent-child relationships for all the input entities:
print("before/after fragment relations:")
for e in zip([(3, 3)] + holes, ovv):
    print("parent " + str(e[0]) + " -> child " + str(e[1]))

gmsh.model.occ.synchronize()

# When the boolean operation leads to simple modifications of entities, and if
# one deletes the original entities, Gmsh tries to assign the same tag to the
# new entities. (This behavior is governed by the
# `Geometry.OCCBooleanPreserveNumbering' option.)

# Here the `Physical Volume' definitions can thus be made for the 5 spheres
# directly, as the five spheres (volumes 4, 5, 6, 7 and 8), which will be
# deleted by the fragment operations, will be recreated identically (albeit with
# new surfaces) with the same tags:
for i in range(1, 6):
    gmsh.model.addPhysicalGroup(3, [3 + i], i)

# The tag of the cube will change though, so we need to access it
# programmatically:
gmsh.model.addPhysicalGroup(3, [ov[-1][1]], 10)

# Creating entities using constructive solid geometry is very powerful, but can
# lead to practical issues for e.g. setting mesh sizes at points, or identifying
# boundaries.

# To identify points or other bounding entities you can take advantage of the
# `getEntities()', `getBoundary()' and `getEntitiesInBoundingBox()' functions:

lcar1 = .1
lcar2 = .0005
lcar3 = .055

# Assign a mesh size to all the points:
gmsh.model.mesh.setSize(gmsh.model.getEntities(0), lcar1)

# Override this constraint on the points of the five spheres:
gmsh.model.mesh.setSize(gmsh.model.getBoundary(holes, False, False, True),
                        lcar3)

# Select the corner point by searching for it geometrically:
eps = 1e-3
ov = gmsh.model.getEntitiesInBoundingBox(0.5 - eps, 0.5 - eps, 0.5 - eps,
                                         0.5 + eps, 0.5 + eps, 0.5 + eps, 0)
gmsh.model.mesh.setSize(ov, lcar2)

gmsh.model.mesh.generate(3)


# Additional examples created with the OpenCASCADE geometry kernel are available
# in `t18.py', `t19.py' and `t20.py', as well as in the `examples/api'
# directory.

# Inspect the log:
# log = gmsh.logger.get()
# print("Logger has recorded " + str(len(log)) + " lines")
# gmsh.logger.stop()

# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

fragment produced volumes:
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
before/after fragment relations:
parent (3, 3) -> child [(3, 9), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8)]
parent (3, 4) -> child [(3, 4)]
parent (3, 5) -> child [(3, 5)]
parent (3, 6) -> child [(3, 6)]
parent (3, 7) -> child [(3, 7)]
parent (3, 8) -> child [(3, 8)]
Info    : Meshing 1D...
Info    : [ 10%] Meshing curve 23 (Circle)
Info    : [ 20%] Meshing curve 26 (Circle)
Info    : [ 20%] Meshing curve 29 (Circle)
Info    : [ 30%] Meshing curve 32 (Circle)
Info    : [ 40%] Meshing curve 35 (Circle)
Info    : [ 50%] Meshing curve 37 (Line)
Info    : [ 50%] Meshing curve 38 (Line)
Info    : [ 50%] Meshing curve 39 (Line)
Info    : [ 50%] Meshing curve 40 (Line)
Info    : [ 60%] Meshing curve 41 (Line)
Info    : [ 60%] Meshing curve 42 (Line)
Info    : [ 60%] Meshing curve 43 (Line)
Info    : [ 70%] Meshing curve 44 (Line)
Info    : [ 70%] Meshing curve 45 (Line)
Info    : [ 70%] Meshing curve 46 (Line)
Info    : [ 70%] 

In [5]:
# Tutorial 18 - Periodic meshes with OpenCASCADE geometry kernel
import gmsh
import math
import sys
import os

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh18"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# a unit cube with a non-uniform mesh size constraint:
gmsh.model.occ.addBox(0, 0, 0, 1, 1, 1, 1)
gmsh.model.occ.synchronize()

gmsh.model.mesh.setSize(gmsh.model.getEntities(0), 0.1)
gmsh.model.mesh.setSize([(0, 1)], 0.02)

# To impose that the mesh on surface 2 (the right side of the cube) should
# match the mesh from surface 1 (the left side), the following periodicity
# constraint is set:
translation = [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
gmsh.model.mesh.setPeriodic(2, [2], [1], translation)

# The periodicity transform is provided as a 4x4 affine transformation matrix,
# given by row.

# During mesh generation, the mesh on surface 2 will be created by copying
# the mesh from surface 1.

# Multiple periodicities can be imposed in the same way:
gmsh.model.mesh.setPeriodic(2, [6], [5],
                            [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1])
gmsh.model.mesh.setPeriodic(2, [4], [3],
                            [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1])

# For more complicated cases, finding the corresponding surfaces by hand can
# be tedious, especially when geometries are created through solid
# modelling. Let's construct a slightly more complicated geometry.

# We start with a cube and some spheres:
gmsh.model.occ.addBox(2, 0, 0, 1, 1, 1, 10)
x = 2 - 0.3
y = 0
z = 0
gmsh.model.occ.addSphere(x, y, z, 0.35, 11)
gmsh.model.occ.addSphere(x + 1, y, z, 0.35, 12)
gmsh.model.occ.addSphere(x, y + 1, z, 0.35, 13)
gmsh.model.occ.addSphere(x, y, z + 1, 0.35, 14)
gmsh.model.occ.addSphere(x + 1, y + 1, z, 0.35, 15)
gmsh.model.occ.addSphere(x, y + 1, z + 1, 0.35, 16)
gmsh.model.occ.addSphere(x + 1, y, z + 1, 0.35, 17)
gmsh.model.occ.addSphere(x + 1, y + 1, z + 1, 0.35, 18)

# We first fragment all the volumes, which will leave parts of spheres
# protruding outside the cube:
out, _ = gmsh.model.occ.fragment([(3, 10)], [(3, i) for i in range(11, 19)])
gmsh.model.occ.synchronize()

# Ask OpenCASCADE to compute more accurate bounding boxes of entities using
# the STL mesh:
gmsh.option.setNumber("Geometry.OCCBoundsUseStl", 1)

# We then retrieve all the volumes in the bounding box of the original cube,
# and delete all the parts outside it:
eps = 1e-3
vin = gmsh.model.getEntitiesInBoundingBox(2 - eps, -eps, -eps, 2 + 1 + eps,
                                          1 + eps, 1 + eps, 3)
for v in vin:
    out.remove(v)
gmsh.model.removeEntities(out, True)  # Delete outside parts recursively

# We now set a non-uniform mesh size constraint (again to check results
# visually):
p = gmsh.model.getBoundary(vin, False, False, True)  # Get all points
gmsh.model.mesh.setSize(p, 0.1)
p = gmsh.model.getEntitiesInBoundingBox(2 - eps, -eps, -eps, 2 + eps, eps, eps,
                                        0)
gmsh.model.mesh.setSize(p, 0.001)

# We now identify corresponding surfaces on the left and right sides of the
# geometry automatically.

# First we get all surfaces on the left:
sxmin = gmsh.model.getEntitiesInBoundingBox(2 - eps, -eps, -eps, 2 + eps,
                                            1 + eps, 1 + eps, 2)

for i in sxmin:
    # Then we get the bounding box of each left surface
    xmin, ymin, zmin, xmax, ymax, zmax = gmsh.model.getBoundingBox(i[0], i[1])
    # We translate the bounding box to the right and look for surfaces inside
    # it:
    sxmax = gmsh.model.getEntitiesInBoundingBox(xmin - eps + 1, ymin - eps,
                                                zmin - eps, xmax + eps + 1,
                                                ymax + eps, zmax + eps, 2)
    # For all the matches, we compare the corresponding bounding boxes...
    for j in sxmax:
        xmin2, ymin2, zmin2, xmax2, ymax2, zmax2 = gmsh.model.getBoundingBox(
            j[0], j[1])
        xmin2 -= 1
        xmax2 -= 1
        # ...and if they match, we apply the periodicity constraint
        if (abs(xmin2 - xmin) < eps and abs(xmax2 - xmax) < eps
                and abs(ymin2 - ymin) < eps and abs(ymax2 - ymax) < eps
                and abs(zmin2 - zmin) < eps and abs(zmax2 - zmax) < eps):
            gmsh.model.mesh.setPeriodic(2, [j[1]], [i[1]], translation)

gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Setting mesh master using transformation
Info    : Setting curve master 8 - 4
Info    : Setting curve master 5 - 1
Info    : Setting curve master 7 - 3
Info    : Setting curve master 6 - 2
Info    : Setting mesh master using transformation
Info    : Setting curve master 12 - 11
Info    : Setting curve master 2 - 4
Info    : Setting curve master 10 - 9
Info    : Setting curve master 6 - 8
Info    : Setting mesh master using transformation
Info    : Setting curve master 12 - 10
Info    : Setting curve master 3 - 1
Info    : Setting curve master 11 - 9
Info    : Setting curve master 7 - 5
Info    : Setting mesh master using transformation
Info    : Setting curve master 52 - 14
Info    : Setting curve master 70 - 13
Info    : Setting curve master 71 - 15
Info    : Setting mesh master using transformation
Info    : Setting curve master 29 - 19
Info    : Setting curve master 54 - 20
Info    : Setting curve master 55 - 22
Info    : Setting curve master 38 - 21
Info    : Setting curv

In [10]:
# Tutorial 19 - Thrusections, fillets, pipes, mesh size from curvature in the OpenCASCADE
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh19"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# `addThruSections()' constructs Volumes from (closed) curve loops
gmsh.model.occ.addCircle(0, 0, 0, 0.5, 1)
gmsh.model.occ.addCurveLoop([1], 1)
gmsh.model.occ.addCircle(0.1, 0.05, 1, 0.1, 2)
gmsh.model.occ.addCurveLoop([2], 2)
gmsh.model.occ.addCircle(-0.1, -0.1, 2, 0.3, 3)
gmsh.model.occ.addCurveLoop([3], 3)
gmsh.model.occ.addThruSections([1, 2, 3], 1)
gmsh.model.occ.synchronize()

# We can also force the creation of ruled surfaces:
gmsh.model.occ.addCircle(2 + 0, 0, 0, 0.5, 11)
gmsh.model.occ.addCurveLoop([11], 11)
gmsh.model.occ.addCircle(2 + 0.1, 0.05, 1, 0.1, 12)
gmsh.model.occ.addCurveLoop([12], 12)
gmsh.model.occ.addCircle(2 - 0.1, -0.1, 2, 0.3, 13)
gmsh.model.occ.addCurveLoop([13], 13)
gmsh.model.occ.addThruSections([11, 12, 13], 11, True, True)
gmsh.model.occ.synchronize()


# We copy the first volume, and fillet all its edges:
out = gmsh.model.occ.copy([(3, 1)])
gmsh.model.occ.translate(out, 4, 0, 0)
gmsh.model.occ.synchronize()
e = gmsh.model.getBoundary(gmsh.model.getBoundary(out), False)
gmsh.model.occ.fillet([out[0][1]], [abs(i[1]) for i in e], [0.1])
gmsh.model.occ.synchronize()

# OpenCASCADE also allows general extrusions along a smooth path. Let's first
# define a spline curve:
nturns = 1.
npts = 20
r = 1.
h = 1. * nturns
p = []
for i in range(0, npts):
    theta = i * 2 * math.pi * nturns / npts
    gmsh.model.occ.addPoint(r * math.cos(theta), r * math.sin(theta),
                            i * h / npts, 1, 1000 + i)
    p.append(1000 + i)
gmsh.model.occ.addSpline(p, 1000)

# A wire is like a curve loop, but open:
gmsh.model.occ.addWire([1000], 1000)

# We define the shape we would like to extrude along the spline (a disk):
gmsh.model.occ.addDisk(1, 0, 0, 0.2, 0.2, 1000)
gmsh.model.occ.rotate([(2, 1000)], 0, 0, 0, 1, 0, 0, math.pi / 2)

# We extrude the disk along the spline to create a pipe (other sweeping types
# can be specified; try e.g. 'Frenet' instead of 'DiscreteTrihedron'):
gmsh.model.occ.addPipe([(2, 1000)], 1000, 'DiscreteTrihedron')

# We delete the source surface, and increase the number of sub-edges for a
# nicer display of the geometry:
gmsh.model.occ.remove([(2, 1000)])
gmsh.option.setNumber("Geometry.NumSubEdges", 1000)

gmsh.model.occ.synchronize()

# We can activate the calculation of mesh element sizes based on curvature
# (here with a target of 20 elements per 2*Pi radians):
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 20)

# We can constraint the min and max element sizes to stay within reasonnable
# values (see `t10.py' for more details):
gmsh.option.setNumber("Mesh.MeshSizeMin", 0.001)
gmsh.option.setNumber("Mesh.MeshSizeMax", 0.3)

gmsh.model.mesh.generate(3)

# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 20%] Meshing curve 4 (BSpline)
Info    : [ 20%] Meshing curve 5 (BSpline)
Info    : [ 30%] Meshing curve 6 (BSpline)
Info    : [ 30%] Meshing curve 11 (Circle)
Info    : [ 40%] Meshing curve 12 (Circle)
Info    : [ 40%] Meshing curve 13 (Circle)
Info    : [ 40%] Meshing curve 14 (BSpline)
Info    : [ 50%] Meshing curve 15 (BSpline)
Info    : [ 50%] Meshing curve 16 (BSpline)
Info    : [ 60%] Meshing curve 17 (BSpline)
Info    : [ 60%] Meshing curve 18 (BSpline)
Info    : [ 70%] Meshing curve 19 (BSpline)
Info    : [ 70%] Meshing curve 20 (BSpline)
Info    : [ 70%] Meshing curve 21 (BSpline)
Info    : [ 80%] Meshing curve 22 (BSpline)
Info    : [ 80%] Meshing curve 1000 (BSpline)
Info    : [ 90%] Meshing curve 1001 (Ellipse)
Info    : [ 90%] Meshing curve 1002 (BSpline)
Info    : [100%] Meshing curve 1003 (BSpline)
Info    : [1

In [12]:
# Tutorial 20 - importing STEP files and to modify/slicing them with OpenCASCADE
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh20"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo

# Load a STEP file (using `importShapes' instead of `merge' allows to directly
# retrieve the tags of the highest dimensional imported entities):
v = gmsh.model.occ.importShapes('./meshes/t20_data.step')

# Get the bounding box of the volume:
xmin, ymin, zmin, xmax, ymax, zmax = gmsh.model.occ.getBoundingBox(
    v[0][0], v[0][1])

# We want to slice the model into N slices, and either keep the volume slices
# or just the surfaces obtained by the cutting:

N = 5  # Number of slices
dir = 'X' # Direction: 'X', 'Y' or 'Z'
surf = False  # Keep only surfaces?

dx = (xmax - xmin)
dy = (ymax - ymin)
dz = (zmax - zmin)
L = dz if (dir == 'X') else dx
H = dz if (dir == 'Y') else dy

# Create the first cutting plane:
s = []
s.append((2, gmsh.model.occ.addRectangle(xmin, ymin, zmin, L, H)))
if dir == 'X':
    gmsh.model.occ.rotate([s[0]], xmin, ymin, zmin, 0, 1, 0, -math.pi/2)
elif dir == 'Y':
    gmsh.model.occ.rotate([s[0]], xmin, ymin, zmin, 1, 0, 0, math.pi/2)
tx = dx / N if (dir == 'X') else 0
ty = dy / N if (dir == 'Y') else 0
tz = dz / N if (dir == 'Z') else 0
gmsh.model.occ.translate([s[0]], tx, ty, tz)

# Create the other cutting planes:
for i in range(1, N-1):
    s.extend(gmsh.model.occ.copy([s[0]]))
    gmsh.model.occ.translate([s[-1]], i * tx, i * ty, i * tz)

# Fragment (i.e. intersect) the volume with all the cutting planes:
gmsh.model.occ.fragment(v, s)

# Now remove all the surfaces (and their bounding entities) that are not on the
# boundary of a volume, i.e. the parts of the cutting planes that "stick out" of
# the volume:
gmsh.model.occ.remove(gmsh.model.occ.getEntities(2), True)

gmsh.model.occ.synchronize()

if surf:
    # If we want to only keep the surfaces, retrieve the surfaces in bounding
    # boxes around the cutting planes...
    eps = 1e-4
    s = []
    for i in range(1, N):
        xx = xmin if (dir == 'X') else xmax
        yy = ymin if (dir == 'Y') else ymax
        zz = zmin if (dir == 'Z') else zmax
        s.extend(gmsh.model.getEntitiesInBoundingBox(
            xmin - eps + i * tx, ymin - eps + i * ty, zmin - eps + i * tz,
            xx + eps + i * tx, yy + eps + i * ty, zz + eps + i * tz, 2))
    # ...and remove all the other entities (here directly in the model, as we
    # won't modify any OpenCASCADE entities later on):
    dels = gmsh.model.getEntities(2)
    for e in s:
        dels.remove(e)
    gmsh.model.removeEntities(gmsh.model.getEntities(3))
    gmsh.model.removeEntities(dels)
    gmsh.model.removeEntities(gmsh.model.getEntities(1))
    gmsh.model.removeEntities(gmsh.model.getEntities(0))

# Finally, let's specify a global mesh size and mesh the partitioned model:
gmsh.option.setNumber("Mesh.MeshSizeMin", 3)
gmsh.option.setNumber("Mesh.MeshSizeMax", 3)
gmsh.model.mesh.generate(3)



# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

Info    :  - Label 'Shapes/Rhino Product' (3D)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (BSpline)
Info    : [ 10%] Meshing curve 2 (BSpline)
Info    : [ 10%] Meshing curve 3 (BSpline)
Info    : [ 10%] Meshing curve 4 (BSpline)
Info    : [ 10%] Meshing curve 5 (BSpline)
Info    : [ 10%] Meshing curve 6 (BSpline)
Info    : [ 10%] Meshing curve 7 (BSpline)
Info    : [ 10%] Meshing curve 8 (BSpline)
Info    : [ 10%] Meshing curve 9 (BSpline)
Info    : [ 10%] Meshing curve 10 (BSpline)
Info    : [ 10%] Meshing curve 11 (BSpline)
Info    : [ 10%] Meshing curve 12 (BSpline)
Info    : [ 10%] Meshing curve 13 (BSpline)
Info    : [ 20%] Meshing curve 14 (BSpline)
Info    : [ 20%] Meshing curve 15 (BSpline)
Info    : [ 20%] Meshing curve 16 (BSpline)
Info    : [ 20%] Meshing curve 17 (BSpline)
Info    : [ 20%] Meshing curve 18 (BSpline)
Info    : [ 20%] Meshing curve 19 (BSpline)
Info    : [ 20%] Meshing curve 20 (BSpline)
Info    : [ 20%] Meshing curve 21 (BSpline)
Info    : [ 20

In [ ]:
# Tutorial -
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

Name = "test_mesh"
gmsh.model.add(Name)

# namespace
gmg = gmsh.model.geo


# saving the mesh
gmsh.write(f"./meshes/{Name}.msh")

# Launch the GUI to see the results:
gmsh.fltk.run()

# finish the session
gmsh.finalize()

In [2]:
# trimesh warwick logo - triangles


import gmsh
import math
import sys
import os

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()


gmsh.model.add("test_mesh_warwick")

# preparing parameters
lx, ly = 1, 1

Lc1 = 0.6     # mesh size 1
Lc2 = 0.003    # mesh size 2


# can redefine a namespace:
gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

# defining points and tagging them
gmg.addPoint(0, 0, 0, Lc1, 1)
gmg.addPoint(lx, 0, 0, Lc1, 2)
gmg.addPoint(lx / 2, -ly, 0, Lc1, 3)
gmg.addPoint(0, -ly / 5, 0, Lc1, 4)
gmg.addPoint(-lx / 2, -ly , 0, Lc1, 5)
gmg.addPoint(-lx, 0 , 0, Lc1, 6)


# adding lines
gmg.addLine(1, 2, 1)
gmg.addLine(2, 3, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 5, 4)
gmg.addLine(5, 6, 5)
gmg.addLine(6, 1, 6)


# We can then define a new surface:
gmg.addCurveLoop([1, 2, 3, 4, 5, 6], 11)
gmg.addPlaneSurface([11], 22)

gmg.synchronize()
# adding physical groups
gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4, 5, 6], 1000)              # curves physical group
gmsh.model.addPhysicalGroup(2, [22], name="My surface")  # surfaces physical group

# synchronize the model
gmg.synchronize()

# To use quadrilateral mesh elements:
#gmsh.model.mesh.setRecombine(2, 22)

# adding some comments by creating a post-processing view
v = gmsh.view.add("comments")


# change the color of some entities:
# gmsh.model.setColor([(2, 22)], 127, 127, 127)  # Gray50
# gmsh.model.setColor([(2, 24)], 160, 32, 240)   # Purple
# gmsh.model.setColor([(1, i) for i in range(1, 15)], 255, 0, 0)     # Red
# gmsh.model.setColor([(1, i) for i in range(15, 21)], 255, 255, 0)  # Yellow
gmsh.model.setColor([(1, i) for i in range(0, 7)], 0, 0, 0)  # black

# global meshing options
gmsh.option.setNumber("Mesh.Algorithm", 6)  # Frontal-Delaunay for 2D meshes
#gmsh.option.setNumber("Mesh.Algorithm", 8)  # Frontal-Delaunay for quadrilaterals

gmsh.model.mesh.generate(2)
gmsh.write("./meshes/warwick_mesh.msh")

# Launch the GUI
gmg.synchronize()
#gmsh.fltk.run()

# finish the session
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Line)
Info    : [ 90%] Meshing curve 6 (Line)
Info    : Done meshing 1D (Wall 0.000524448s, CPU 0.000776s)
Info    : Meshing 2D...
Info    : Meshing surface 22 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.000458086s, CPU 0.000437s)
Info    : 14 nodes 32 elements
Info    : Writing './meshes/warwick_mesh.msh'...
Info    : Done writing './meshes/warwick_mesh.msh'


In [1]:
# trimesh warwick logo - quadliterals


import gmsh
import math
import sys
import os

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()


gmsh.model.add("test_mesh_warwick")

# preparing parameters
lx, ly = 1, 1

Lc1 = 0.6     # mesh size 1
Lc2 = 0.003    # mesh size 2


# can redefine a namespace:
gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

# defining points and tagging them
gmg.addPoint(0, 0, 0, Lc1, 1)
gmg.addPoint(lx, 0, 0, Lc1, 2)
gmg.addPoint(lx / 2, -ly, 0, Lc1, 3)
gmg.addPoint(0, -ly / 5, 0, Lc1, 4)
gmg.addPoint(-lx / 2, -ly , 0, Lc1, 5)
gmg.addPoint(-lx, 0 , 0, Lc1, 6)


# adding lines
gmg.addLine(1, 2, 1)
gmg.addLine(2, 3, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 5, 4)
gmg.addLine(5, 6, 5)
gmg.addLine(6, 1, 6)


# We can then define a new surface:
gmg.addCurveLoop([1, 2, 3, 4, 5, 6], 11)
gmg.addPlaneSurface([11], 22)

# adding physical groups
gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4, 5, 6], 1000) # curves physical group
gmsh.model.addPhysicalGroup(2, [22], name="My surface")  # surfaces physical group


# ftion 'setTransfiniteCurve()' explicitly specifies location of mesh nodes on the curve.
# gmg.mesh.setTransfiniteCurve(1, 4)  # 6 uniformly distributed points
# gmg.mesh.setTransfiniteCurve(2, 6)  # 6 uniform points
# gmg.mesh.setTransfiniteCurve(3, 4)  # 4 uniform points
# gmg.mesh.setTransfiniteCurve(4, 4)  # 4 uniform points
# gmg.mesh.setTransfiniteCurve(5, 4)  # 5 uniform points
# gmg.mesh.setTransfiniteCurve(6, 6)  # 5 uniform points
#
#
# # geometric series location of the mesh
# #gmg.mesh.setTransfiniteCurve(1, 30, "Progression", -1.2)  # geom. distribution (reversed)
# #gmg.mesh.setTransfiniteCurve(3, 30, "Progression", 1.2)   # geom. distribution
#
# # meshing the surface
# gmg.mesh.setTransfiniteSurface(1, "Left", [1,2,3,4,5,6])  # set up the corners of interpolation



# if there are some triangles left - 2 methods:
gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 3) # coarses recombination (2 or 3)
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)   # subdivision step alone




# synchronize the model
gmg.synchronize()

# To use quadrilateral mesh elements:
gmsh.model.mesh.setRecombine(2, 22)

# adding some comments by creating a post-processing view
v = gmsh.view.add("comments")


# change the color of some entities:
# gmsh.model.setColor([(2, 22)], 127, 127, 127)  # Gray50
# gmsh.model.setColor([(2, 24)], 160, 32, 240)   # Purple
# gmsh.model.setColor([(1, i) for i in range(1, 15)], 255, 0, 0)     # Red
# gmsh.model.setColor([(1, i) for i in range(15, 21)], 255, 255, 0)  # Yellow
gmsh.model.setColor([(1, i) for i in range(0, 7)], 0, 0, 0)  # black

# global meshing options
gmsh.option.setNumber("Mesh.Algorithm", 8)  # Frontal-Delaunay for 2D meshes
#gmsh.option.setNumber("Mesh.Algorithm", 8)  # Frontal-Delaunay for quadrilaterals

gmsh.model.mesh.generate(2)


# if we want to recombine the mesh after generating it, we use mesh.refine()
# gmsh.model.mesh.generate(2)
# gmsh.model.mesh.recombine()
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
# gmsh.model.mesh.refine()
#gmsh.write("test_mesh4.msh")

# Launch the GUI
gmg.synchronize()
gmsh.fltk.run()

# finish the session
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Line)
Info    : [ 90%] Meshing curve 6 (Line)
Info    : Done meshing 1D (Wall 0.000444942s, CPU 0.001492s)
Info    : Meshing 2D...
Info    : Meshing surface 22 (Plane, Frontal-Delaunay for Quads)
Info    : Blossom: 3 internal 4 closed
Info    : Blossom recombination completed (Wall 7.7887e-05s, CPU 0.00015s): 2 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.355615, min Q = 0.355615
Info    : Blossom recombination completed (Wall 0.000246667s, CPU 0.000716s): 8 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.588928, min Q = 0.355615
Info    : Done meshing 2D (Wall 0.000749364s, CPU 0.002043s)
Info    : 15 nodes 26 elements
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Pu

In [3]:
# trimesh warwick logo - truss


import gmsh
import math
import sys
import os

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()


gmsh.model.add("test_mesh_warwick")

# preparing parameters
lx, ly = 1, 1

Lc1 = 0.2     # mesh size 1
Lc2 = 0.003    # mesh size 2


# can redefine a namespace:
gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

# defining points and tagging them
gmg.addPoint(0, 0, 0, Lc1, 1)
gmg.addPoint(lx, 0, 0, Lc1, 2)
gmg.addPoint(lx / 2, -ly, 0, Lc1, 3)
gmg.addPoint(0, -ly / 5, 0, Lc1, 4)
gmg.addPoint(-lx / 2, -ly , 0, Lc1, 5)
gmg.addPoint(-lx, 0 , 0, Lc1, 6)


# adding lines
gmg.addLine(1, 2, 1)
gmg.addLine(2, 3, 2)
gmg.addLine(3, 4, 3)
gmg.addLine(4, 5, 4)
gmg.addLine(5, 6, 5)
gmg.addLine(6, 1, 6)

gmg.addLine(4, 1, 7)
gmg.addLine(6, 4, 8)
gmg.addLine(4, 2, 9)


# We can then define a new surface:
gmg.addCurveLoop([1, 2, 3, 4, 5, 6], 11)
gmg.addPlaneSurface([11], 22)
gmg.synchronize()


# adding physical groups
gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4, 5, 6], 1000) # curves physical group
gmsh.model.addPhysicalGroup(2, [22], name="My surface")  # surfaces physical group


# ftion 'setTransfiniteCurve()' explicitly specifies location of mesh nodes on the curve.
# gmg.mesh.setTransfiniteCurve(1, 4)  # 6 uniformly distributed points
# gmg.mesh.setTransfiniteCurve(2, 6)  # 6 uniform points
# gmg.mesh.setTransfiniteCurve(3, 4)  # 4 uniform points
# gmg.mesh.setTransfiniteCurve(4, 4)  # 4 uniform points
# gmg.mesh.setTransfiniteCurve(5, 4)  # 5 uniform points
# gmg.mesh.setTransfiniteCurve(6, 6)  # 5 uniform points
#
#
# # geometric series location of the mesh
# #gmg.mesh.setTransfiniteCurve(1, 30, "Progression", -1.2)  # geom. distribution (reversed)
# #gmg.mesh.setTransfiniteCurve(3, 30, "Progression", 1.2)   # geom. distribution
#
# # meshing the surface
# gmg.mesh.setTransfiniteSurface(1, "Left", [1,2,3,4,5,6])  # set up the corners of interpolation



# if there are some triangles left - 2 methods:
#gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 3) # coarses recombination (2 or 3)
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)   # subdivision step alone




# synchronize the model
gmg.synchronize()

# To use quadrilateral mesh elements:
gmsh.model.mesh.setRecombine(2, 22)

# change the color of some entities:
#gmsh.model.setColor([(2, 22)], 127, 127, 127)  # Gray50
# gmsh.model.setColor([(2, 24)], 160, 32, 240)   # Purple
# gmsh.model.setColor([(1, i) for i in range(1, 15)], 255, 0, 0)     # Red
gmsh.model.setColor([(1, i) for i in range(0, 10)], 0, 0, 0)  # black
gmsh.model.setColor([(2, 11)], 160, 32, 240)   # Purple

# global meshing options
gmsh.option.setNumber("Mesh.Algorithm", 8)  # Frontal-Delaunay for 2D meshes
#gmsh.option.setNumber("Mesh.Algorithm", 8)  # Frontal-Delaunay for quadrilaterals

#gmsh.model.mesh.generate(2)


# if we want to recombine the mesh after generating it, we use mesh.refine()
# gmsh.model.mesh.generate(2)
# gmsh.model.mesh.recombine()
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
# gmsh.model.mesh.refine()
#gmsh.write("test_mesh4.msh")

# Launch the GUI
gmg.synchronize()
gmsh.fltk.run()

# finish the session
gmsh.finalize()


-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.1
Packaged by   : runner
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [14]:
# This reimplements gmsh/examples/boolean/boolean.geo in Python.

import gmsh
import sys

gmsh.initialize(sys.argv)

gmsh.model.add("boolean")

# from http://en.wikipedia.org/wiki/Constructive_solid_geometry

gmsh.option.setNumber("Mesh.Algorithm", 6)
gmsh.option.setNumber("Mesh.MeshSizeMin", 0.4)
gmsh.option.setNumber("Mesh.MeshSizeMax", 0.4)

R = 1.4
Rs = R * .7
Rt = R * 1.25

gmsh.model.occ.addBox(-R, -R, -R, 2 * R, 2 * R, 2 * R, 1)
gmsh.model.occ.addSphere(0, 0, 0, Rt, 2)
gmsh.model.occ.intersect([(3, 1)], [(3, 2)], 3)
gmsh.model.occ.addCylinder(-2 * R, 0, 0, 4 * R, 0, 0, Rs, 4)
gmsh.model.occ.addCylinder(0, -2 * R, 0, 0, 4 * R, 0, Rs, 5)
gmsh.model.occ.addCylinder(0, 0, -2 * R, 0, 0, 4 * R, Rs, 6)
gmsh.model.occ.fuse([(3, 4), (3, 5)], [(3, 6)], 7)
gmsh.model.occ.cut([(3, 3)], [(3, 7)], 8)

gmsh.model.occ.synchronize()

gmsh.model.mesh.generate(3)
#gmsh.model.mesh.refine()
#gmsh.model.mesh.setOrder(2)
#gmsh.model.mesh.partition(4)

#gmsh.write("boolean.msh")

# Launch the GUI
gmg.synchronize()
gmsh.fltk.run()

gmsh.finalize()


Error   : Unknown mesh format '/Users/macbookpro/Library/Jupyter/runtime/kernel-485f03c6-cee6-4669-9b4a-39fc77383389.json'


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 10%] Meshing curve 4 (Circle)
Info    : [ 10%] Meshing curve 5 (Circle)
Info    : [ 20%] Meshing curve 6 (Circle)
Info    : [ 20%] Meshing curve 7 (Circle)
Info    : [ 20%] Meshing curve 8 (Circle)
Info    : [ 20%] Meshing curve 9 (Circle)
Info    : [ 30%] Meshing curve 10 (Circle)
Info    : [ 30%] Meshing curve 11 (Line)
Info    : [ 30%] Meshing curve 12 (Ellipse)
Info    : [ 30%] Meshing curve 13 (Ellipse)
Info    : [ 40%] Meshing curve 14 (Ellipse)
Info    : [ 40%] Meshing curve 15 (Ellipse)
Info    : [ 40%] Meshing curve 16 (Ellipse)
Info    : [ 40%] Meshing curve 17 (Circle)
Info    : [ 50%] Meshing curve 18 (Circle)
Info    : [ 50%] Meshing curve 19 (Circle)
Info    : [ 50%] Meshing curve 20 (Circle)
Info    : [ 50%] Meshing curve 21 (Circle)
Info    : [ 60%] Meshing curve 22 (Ellipse)
Info    : [ 60%] Meshing curve 23 (

In [16]:
# Tutorial - Surface filling and BSplines
# (from https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/examples/api/surface_filling.py)
import gmsh

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

gmsh.option.setNumber('Mesh.MeshSizeMin', 0.02)
gmsh.option.setNumber('Mesh.MeshSizeMax', 0.02)
gmsh.model.occ.addPoint(-0.8, 0.1, -0.2, 1)
gmsh.model.occ.addPoint(-0.5, 0.1, -0.2, 2)
gmsh.model.occ.addPoint(-0.6, 0, -0.1, 3)
gmsh.model.occ.addPoint(-0.7, -0, -0.1, 4)
gmsh.model.occ.addPoint(-0.7, 0.2, -0.2, 5)
gmsh.model.occ.addPoint(-0.6, 0.1, -0.1, 6)
gmsh.model.occ.addSpline([1, 5, 2], 1)
gmsh.model.occ.addSpline([2, 6, 3], 2)
gmsh.model.occ.addSpline([3, 4, 1], 3)
gmsh.model.occ.addCurveLoop([1, 2, 3], 1)

# Bspline surface bounded by curve loop 1, constructed by optimization
gmsh.model.occ.addSurfaceFilling(1, 1)

# BSpline filling (try "Stretch" or "Curved")
gmsh.model.occ.addBSplineFilling(1, 2, 'Curved')

# Bezier filling - can be used if all bounding curves are Bezier curves
# gmsh.model.occ.addBezierFilling(1, 3)

# Same as 1, but passing through points 7 and 8
gmsh.model.occ.addPoint(-0.7, 0.1, -0.2, 7)
gmsh.model.occ.addPoint(-0.67, 0.1, -0.2, 8)
gmsh.model.occ.addSurfaceFilling(1, 4, [7, 8])

gmsh.model.occ.synchronize()

gmsh.fltk.run()

gmsh.finalize()


-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.1
Packaged by   : runner
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [21]:
# Tutorial - multi-threading
# (from https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/examples/api/multi_thread.py)
from multiprocessing import Process
import gmsh

# to mesh independent entities you can of course run multiple independent Gmsh
# processes as well (cf. multi_thread.py for actual parallel mesh generation)

import gmsh

# when Gmsh is compiled with OpenMP support the meshing pipeline is
# multi-threaded; the number of threads is governed by the "General.NumThreads"
# option (cf. multi_process.py for parallel meshing of independent models)

gmsh.initialize()

for i in range(5):
    s = gmsh.model.occ.addRectangle(i,0,0, 1,1)

gmsh.model.occ.synchronize()
gmsh.option.setNumber('Mesh.MeshSizeMax', 0.005)
gmsh.option.setNumber('General.NumThreads', 5)
gmsh.model.mesh.generate(2)
gmsh.finalize()
print("All done")



Info    : Meshing 1D...
Info    : [  0%] Meshing curve 17 (Line)
Info    : [  0%] Meshing curve 13 (Line)
Info    : [  0%] Meshing curve 1 (Line)
Info    : [  0%] Meshing curve 5 (Line)
Info    : [  0%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 20%] Meshing curve 10 (Line)
Info    : [ 30%] Meshing curve 14 (Line)
Info    : [ 30%] Meshing curve 18 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 11 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 15 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 60%] Meshing curve 19 (Line)
Info    : [ 70%] Meshing curve 12 (Line)
Info    : [ 70%] Meshing curve 16 (Line)
Info    : [ 80%] Meshing curve 20 (Line)
Info    : Done meshing 1D (Wall 0.00258748s, CPU 0.007569s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [  0%] Mes

In [23]:
# Tutorial - Finding Neighbours
# (from https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/examples/api/neighbors.py )
import gmsh
import sys

# small example showing how the api can be used to compute the neighbours (by a
# face) of all tets in the mesh

gmsh.initialize()

gmsh.model.add("my test model")
gmsh.model.occ.addBox(0, 0, 0, 1, 1, 1)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

print("--- getting tets and face nodes")
tets, _ = gmsh.model.mesh.getElementsByType(4)
fnodes = gmsh.model.mesh.getElementFaceNodes(4, 3)

print("--- computing face x tet incidence")
faces = []
fxt = {}
for i in range(0, len(fnodes), 3):
    f = tuple(sorted(fnodes[i:i + 3]))
    faces.append(f)
    t = tets[i // 12]
    if not f in fxt:
        fxt[f] = [t]
    else:
        fxt[f].append(t)

print("--- computing neighbors by face")
txt = {}
for i in range(0, len(faces)):
    f = faces[i]
    t = tets[i // 4]
    if not t in txt:
        txt[t] = set()
    for tt in fxt[f]:
        if tt != t:
            txt[t].add(tt)

print("--- done: neighbors by face =", txt)

gmsh.model.occ.synchronize()

gmsh.fltk.run()

gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00160554s, CPU 0.001767s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0132843s,

In [25]:
# Tutorial - Normals
# (from  https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/examples/api/normals.py )
import gmsh
import sys

gmsh.initialize()
gmsh.model.add("normals")
gmsh.model.occ.addSphere(0, 0, 0, 1)
gmsh.model.occ.addBox(2, 0, 0, 1, 1, 1)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(2)

nn = []
cc = []

# get all surfaces
ent = gmsh.model.getEntities(2)

for e in ent:
    surf = e[1]
    # get nodes on surf, including those on the boundary (contrary to internal
    # nodes, which store their parametric coordinates, boundary nodes will be
    # reparametrized on surf in order to compute their parametric coordinates -
    # which will be different when reparametrized on another adjacent surface)
    tags, coord, param = gmsh.model.mesh.getNodes(2, surf, True)
    # get surface normal on all nodes, i.e. including on the geometrical
    # singularities (edges/points)
    normals = gmsh.model.getNormal(surf, param)
    # get surface curvature
    curv = gmsh.model.getCurvature(2, surf, param)
    for i in range(0, len(coord), 3):
        nn.append(coord[i])
        nn.append(coord[i + 1])
        nn.append(coord[i + 2])
        nn.append(normals[i])
        nn.append(normals[i + 1])
        nn.append(normals[i + 2])
        cc.append(coord[i])
        cc.append(coord[i + 1])
        cc.append(coord[i + 2])
        cc.append(curv[i // 3])

t = gmsh.view.add("normals")
gmsh.view.addListData(t, "VP", len(nn) // 6, nn)
#gmsh.view.write(t, "normals.pos")

t = gmsh.view.add("curvatures")
gmsh.view.addListData(t, "SP", len(cc) // 4, cc)
#gmsh.view.write(t, "curvatures.pos")

gmsh.model.occ.synchronize()

gmsh.fltk.run()

gmsh.finalize()


Info    : Meshing 1D...
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 40%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 50%] Meshing curve 8 (Line)
Info    : [ 60%] Meshing curve 9 (Line)
Info    : [ 60%] Meshing curve 10 (Line)
Info    : [ 70%] Meshing curve 11 (Line)
Info    : [ 80%] Meshing curve 12 (Line)
Info    : [ 80%] Meshing curve 13 (Line)
Info    : [ 90%] Meshing curve 14 (Line)
Info    : [100%] Meshing curve 15 (Line)
Info    : Done meshing 1D (Wall 0.00114718s, CPU 0.001534s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Sphere, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 6 (Plane, Frontal-Delauna

In [29]:
# Tutorial - Partitioning Entities
# (from https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/examples/api/partition.py )
import gmsh

write_file = False
write_one_file_per_partition = False
partition_using_metis = False

gmsh.initialize()

# create a simple geometry and mesh it
gmsh.model.add("test")
gmsh.model.occ.addRectangle(0, 0, 0, 1, 1)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(2)

# partition the mesh using Metis, or using the SimplePartition plugin (to create
# simple chessboard-like partitions). This will create new ("partitioned")
# entities in the model, that will behave exactly like other model entities. In
# particular, the full boundary representation is constructed provided that
# Mesh.PartitionCreateTopology == 1. The only difference is that partitioned
# entities have a "parent", which allows to link the partitioned entity with the
# entity it is a subset of. There are other options to govern how physical
# groups are treated (Mesh.PartitionCreatePhysicals), and if ghost cells should
# be created (Mesh.PartitionCreateGhostCells).
if partition_using_metis:
    gmsh.model.mesh.partition(3)
else:
    gmsh.plugin.setNumber("SimplePartition", "NumSlicesX", 3.)
    gmsh.plugin.run("SimplePartition")

# write the partitioned mesh to disk?
if write_file:
    # create one file per partition?
    if write_one_file_per_partition:
        gmsh.option.setNumber("Mesh.PartitionSplitMeshFiles", 1)
    gmsh.write("partition.msh")

# iterate over partitioned entities and print some info
entities = gmsh.model.getEntities()
for e in entities:
    partitions = gmsh.model.getPartitions(e[0], e[1])
    if len(partitions):
        print("Entity " + str(e) + " of type " +
              gmsh.model.getType(e[0], e[1]))
        print(" - Partition(s): " + str(partitions))
        print(" - Parent: " + str(gmsh.model.getParent(e[0], e[1])))
        print(" - Boundary: " + str(gmsh.model.getBoundary([e])))

gmsh.fltk.run()

gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000489574s, CPU 0.000772s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00949553s, CPU 0.008931s)
Info    : 98 nodes 198 elements
Info    : Running Plugin(SimplePartition)...
Info    : Creating partition topology...
Info    :  - Creating partition curves
Info    :  - Creating partition points
Info    : Done creating partition topology
Info    : Done running Plugin(SimplePartition)
Entity (0, 5) of type Partition point
 - Partition(s): [1]
 - Parent: (0, 1)
 - Boundary: []
Entity (0, 6) of type Partition point
 - Partition(s): [30]
 - Parent: (0, 2)
 - Boundary: []
Entity (0, 7) of type Partition point
 - Partition(s): [30]
 - Parent: (0, 3)
 - Boundary: []
Entity (0, 8) of type Partition point
 - Partition

In [43]:
# -----------------------------------------------------------------------------
#
#  Gmsh Python extended tutorial 7
#
#  Additional mesh data: internal edges and faces
#
# -----------------------------------------------------------------------------

import sys
import gmsh

gmsh.initialize()

gmsh.model.add("x7")

# Meshes are fully described in Gmsh by nodes and elements, both associated to
# model entities. The API can be used to generate and handle other mesh
# entities, i.e. mesh edges and faces, which are not stored by default.

lc = 0.5

# Let's create a simple model and mesh it:
gmsh.model.occ.addPoint(0, 0, 0, lc, 1)
gmsh.model.occ.addPoint(0, 8, 0, lc, 2)
gmsh.model.occ.addPoint(8, 8, 0, lc, 3)
gmsh.model.occ.addPoint(8, 0, 0, lc, 4)
gmsh.model.occ.synchronize()

gmsh.model.occ.addLine(1, 2, 1)
gmsh.model.occ.addLine(2, 3, 2)
gmsh.model.occ.addLine(3, 4, 3)
gmsh.model.occ.addLine(4, 1, 4)
gmsh.model.occ.synchronize()

gmsh.model.occ.addCurveLoop([1, 2, 3, 4], 1)
gmsh.model.occ.addPlaneSurface([1], 10)

gmsh.option.setNumber("Mesh.MeshSizeMin", 2.)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(2)

# Like elements, mesh edges and faces are described by (an ordered list of)
# their nodes. Let us retrieve the edges and the (triangular) faces of all the
# first order tetrahedra in the mesh:
elementType = gmsh.model.mesh.getElementType("Triangle", 1)
edgeNodes = gmsh.model.mesh.getElementEdgeNodes(elementType)
faceNodes = gmsh.model.mesh.getElementFaceNodes(elementType, 3)

# Edges and faces are returned for each element as a list of nodes corresponding
# to the canonical orientation of the edges and faces for a given element type.

# Gmsh can also identify unique edges and faces (a single edge or face whatever
# the ordering of their nodes) and assign them a unique tag. This identification
# can be done internally by Gmsh (e.g. when generating keys for basis
# functions), or requested explicitly as follows:
gmsh.model.mesh.createEdges()
gmsh.model.mesh.createFaces()

# Edge and face tags can then be retrieved by providing their nodes:
edgeTags, edgeOrientations = gmsh.model.mesh.getEdges(edgeNodes)
faceTags, faceOrientations = gmsh.model.mesh.getFaces(3, faceNodes)

# Since element edge and face nodes are returned in the same order as the
# elements, one can easily keep track of which element(s) each edge or face is
# connected to:
elementTags, elementNodeTags = gmsh.model.mesh.getElementsByType(elementType)
edges2Elements = {}
faces2Elements = {}
for i in range(len(edgeTags)): # 6 edges per tetrahedron
    if not edgeTags[i] in edges2Elements:
        edges2Elements[edgeTags[i]] = [elementTags[i // 6]]
    else:
        edges2Elements[edgeTags[i]].append(elementTags[i // 6])
for i in range(len(faceTags)): # 4 faces per tetrahedron
    if not faceTags[i] in faces2Elements:
        faces2Elements[faceTags[i]] = [elementTags[i // 4]]
    else:
        faces2Elements[faceTags[i]].append(elementTags[i // 4])

# New unique lower dimensional elements can also be easily created given the
# edge or face nodes. This is especially useful for numerical methods that
# require integrating or interpolating on internal edges or faces (like
# e.g. Discontinuous Galerkin techniques), since creating elements for the
# internal entities will make this additional mesh data readily available (see
# `x6.py'). For example, we can create a new discrete surface...
s = gmsh.model.addDiscreteEntity(2)

# ... and fill it with unique triangles corresponding to the faces of the
# tetrahedra:
maxElementTag = gmsh.model.mesh.getMaxElementTag()
uniqueFaceTags = set()
tagsForTriangles = []
faceNodesForTriangles = []
for i in range(len(faceTags)):
    if faceTags[i] not in uniqueFaceTags:
        uniqueFaceTags.add(faceTags[i])
        tagsForTriangles.append(faceTags[i] + maxElementTag)
        faceNodesForTriangles.append(faceNodes[3 * i])
        faceNodesForTriangles.append(faceNodes[3 * i + 1])
        faceNodesForTriangles.append(faceNodes[3 * i + 2])
elementType2D = gmsh.model.mesh.getElementType("triangle", 1)
gmsh.model.mesh.addElementsByType(s, elementType2D, tagsForTriangles,
                                  faceNodesForTriangles)

# Since the tags for the triangles have been created based on the face tags,
# the information about neighboring elements can also be readily created,
# useful e.g. in Finite Volume or Discontinuous Galerkin techniques:
for t in tagsForTriangles:
    print("triangle " + str(int(t)) + " is connected to tetrahedra " +
          str(faces2Elements[t - maxElementTag]))

# If all you need is the list of all edges or faces in terms of their nodes, you
# can also directly call:
edgeTags, edgeNodes = gmsh.model.mesh.getAllEdges()
faceTags, faceNodes = gmsh.model.mesh.getAllFaces(3)

# Launch the GUI
gmsh.fltk.run()

gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000597492s, CPU 0.000906s)
Info    : Meshing 2D...
Info    : Meshing surface 10 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00343986s, CPU 0.003447s)
Info    : 30 nodes 62 elements
triangle 126 is connected to tetrahedra [21]
triangle 127 is connected to tetrahedra [21]
triangle 128 is connected to tetrahedra [21]
triangle 129 is connected to tetrahedra [21]
triangle 130 is connected to tetrahedra [22]
triangle 131 is connected to tetrahedra [22]
triangle 132 is connected to tetrahedra [22]
triangle 133 is connected to tetrahedra [22]
triangle 134 is connected to tetrahedra [23]
triangle 135 is connected to tetrahedra [23]
triangle 136 is connected to tetrahedra [23]
triangle 137 is connected to tetrahedra [23]
triangle 138 is connected to tetrahedra [24]
